# Drive

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd "drive/MyDrive/Colab Notebooks/mest"
# %ls

In [3]:
# FOLDER = "Zhukovsky/IAM_lines"
# !zip -r "IAM_lines.zip" "{FOLDER}"

In [4]:
ls

Natalia/  SK_recognition/  Untitled.ipynb  roms/


In [5]:
cd Natalia

/data/Natalia


/home/admin/miniconda3/envs/science/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
ls

info/  КККМFOLDER_bin/        КККМFOLDER_strokes_bin_ok/
logs/  КККМFOLDER_bin.tar.xz  КККМFOLDER_strokes_bin_ok.tar.xz


# Библиотеки

In [7]:
import os
import pickle
from collections import defaultdict

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import random
sns.set_theme()

import torch

from tqdm.notebook import tqdm as tqdm_n
from tqdm import tqdm
from functools import partial
from time import gmtime, strftime, time

from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix

from torchvision.transforms import v2
from torchvision.utils import save_image

In [8]:
!nvidia-smi

Sun May  4 22:25:17 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:5E:00.0 Off |                  N/A |
| 30%   33C    P8              27W / 350W |     12MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Code

## My_Dataset

In [9]:
# https://pytorch.org/vision/main/generated/torchvision.datasets.ImageFolder.html

class My_Folder(Dataset):
    def __init__(self, folder, transform, topk, skipNoName=True):
        super().__init__()

        self.folder = folder
        self.transform = transform

        self.dir_names = sorted(os.listdir(folder), reverse=True,
                                key=lambda i: len(os.listdir(os.path.join(folder, i)))) # ["000" , "001", ...]

        # self.dir_names = sorted(os.listdir(folder), reverse=False,
        #                         key=lambda i: len(os.listdir(os.path.join(folder, i))))

        if skipNoName and "Неустановленное лицо" in self.dir_names:
             self.dir_names.remove("Неустановленное лицо")

        if topk:
            self.dir_names = self.dir_names[:topk]

        self.num_cls = len(self.dir_names)

        self.dir2cls = {dir : i for i, dir in enumerate(self.dir_names)}
        self.cls2dir = {cls : dir for dir, cls in self.dir2cls.items()}

        self.dataset = [] # список всех путей до изображений

        self.cls2paths = defaultdict(list) # по cls получить все изображения класса

        for dir in self.dir_names:
            dir_path = os.path.join(folder, dir)

            for i in sorted(os.listdir(dir_path)):
                image_path = os.path.join(dir_path, i)
                cls = self.dir2cls[dir]

                self.dataset.append((image_path, cls))
                self.cls2paths[cls].append(image_path)

        self.cls2count = {cls : len(idxs) for cls, idxs in self.cls2paths.items()}
        self.dir2count = {self.cls2dir[cls] : count for cls, count in self.cls2count.items()}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image_path, cls = self.dataset[idx]
        image = self.transform(Image.open(image_path).convert("RGB"))

        return image, cls


class My_TripletFolder(My_Folder):
    def __getitem__(self, idx):
        anchor_path, cls = self.dataset[idx]
        anchor = self.transform(Image.open(anchor_path).convert("RGB"))

        positive_path = random.choice(self.cls2paths[cls])
        positive = self.transform(Image.open(positive_path).convert("RGB"))

        negative_cls = random.choice([i for i in range(self.num_cls) if i != cls])
        negative_path = random.choice(self.cls2paths[negative_cls])
        negative = self.transform(Image.open(negative_path).convert("RGB"))

        return anchor, positive, negative

In [10]:
class sub_Dataset(Dataset):
    def __init__(self, cls2dir, dir2paths, transform):
        super().__init__()

        # self.cls2dir = {cls : dir for cls, dir in cls2dir.items() if dir in dir2paths} # вдруг папка dir пустая
        self.cls2dir = {cls : dir for cls, dir in cls2dir.items()}

        self.dir2paths = dir2paths
        self.transform = transform

        self.dir2cls = {dir : cls for cls, dir in self.cls2dir.items()}
        # print(self.dir2cls)
        self.cls2paths = {self.dir2cls[dir] : paths for dir, paths in self.dir2paths.items()}

        self.path2cls = defaultdict(int)
        for cls, paths in self.cls2paths.items():
            for path in paths:
                self.path2cls[path] = cls

        self.classes = [cls for cls in self.cls2dir]

        # self.cls2count = {cls : len(idxs) for cls, idxs in self.cls2paths.items()}
        self.dir2count = {}
        for dir in self.cls2dir.values():
            self.dir2count[dir] = len(dir2paths[dir]) if dir in dir2paths else 0


        self.len_dataset = sum(self.dir2count.values())
        # self.dir2count = {self.cls2dir[cls] : count for cls, count in self.cls2count.items()}
        self.cls2count = {self.dir2cls[dir] : count for dir, count in self.dir2count.items()}

    def __len__(self):
        return self.len_dataset

    def __getitem__(self, idx):
        if idx >= self.len_dataset:
            raise IndexError

        cls = random.choice(self.classes)
        image_path = random.choice(self.cls2paths[cls])

        image = self.transform(Image.open(image_path).convert("RGB"))
        # image = None
        return image, cls


class sub_TripletDataset(sub_Dataset):
    def __getitem__(self, idx):
        cls = random.choice(self.classes)

        anchor_path = random.choice(self.cls2paths[cls])
        anchor = self.transform(Image.open(anchor_path).convert("RGB"))

        positive_path = random.choice(self.cls2paths[cls])
        positive = self.transform(Image.open(positive_path).convert("RGB"))

        negative_cls = random.choice([i for i in self.classes if i != cls])
        negative_path = random.choice(self.cls2paths[negative_cls])
        negative = self.transform(Image.open(negative_path).convert("RGB"))

        return anchor, positive, negative


def pre_init_dataset(folder, topk, skipNoName=True):
    dir_names = sorted(os.listdir(folder), reverse=True,
                            key=lambda i: len(os.listdir(os.path.join(folder, i)))) # ["000" , "001", ...]

    # dir_names = sorted(os.listdir(folder), reverse=False,
    #                         key=lambda i: len(os.listdir(os.path.join(folder, i)))) # ["000" , "001", ...]

    if skipNoName and "Неустановленное лицо" in dir_names:
            dir_names.remove("Неустановленное лицо")

    if topk:
        dir_names = dir_names[:topk]

    num_cls = len(dir_names)

    cls2dir = {i : dir for i, dir in enumerate(dir_names)}

    dir2paths = defaultdict(list) # по cls получить все изображения класса

    for dir in dir_names:
        dir_path = os.path.join(folder, dir)

        for i in sorted(os.listdir(dir_path)):
            image_path = os.path.join(dir_path, i)

            dir2paths[dir].append(image_path)

    return cls2dir, dir2paths


def split_dir2paths(dir2paths, percent):
    train_dir2paths, test_dir2paths = defaultdict(list), defaultdict(list)

    for dir, paths in dir2paths.items():
        num = len(paths)
        train_size = int(num * percent)

        flags = [i < train_size for i in range(len(paths))]
        random.shuffle(flags)
        # flags = [random.random() < percent for _ in paths]

        for i, f in enumerate(flags):
            if f:
                train_dir2paths[dir].append(paths[i])
            else:
                test_dir2paths[dir].append(paths[i])

    return train_dir2paths, test_dir2paths


def My_Balanced_split_dataset(folder, transform, topk, skipNoName=True, percent=0.8, type="siam"):
    cls2dir, dir2paths = pre_init_dataset(folder, topk, skipNoName)

    train_dir2paths, test_dir2paths = split_dir2paths(dir2paths, percent)

    if type == "siam":
        class_dataset = sub_TripletDataset
    else:
        class_dataset = sub_Dataset

    return class_dataset(cls2dir, train_dir2paths, transform), class_dataset(cls2dir, test_dir2paths, transform)

## Files

In [11]:
def my_round(x, n):
    new_x = f"{{:.{n-1}g}}".format(x)

    if len(new_x) < n:
        new_x = new_x + " " * (n - len(new_x))
    return new_x


def namefile(path):
        return path.split("/")[-1]


def recur_paths(dir):
    ans = []

    for img in os.listdir(dir):
        x = os.path.join(dir, img)
        if os.path.isdir(x):
            ans.extend(recur_paths(x))
        else:
            ans.append(x)
    return ans


def save_logs(logs, path):
    with open(path, "wb") as f:
        pickle.dump(logs, f, protocol=pickle.HIGHEST_PROTOCOL)


def load_logs(path):
    with open(path, "rb") as f:
        logs = pickle.load(f)

    return logs


def write2file(file, text):
    with open(file, 'a') as f:
        f.write(text + "\n")


def my_time():
    return strftime("%Y-%m-%d %H:%M:%S", gmtime(time() + 3 * 60 * 60))


def my_timediff(start_time):
    return strftime("%H:%M:%S", gmtime(time() - start_time))

## Confusion matrix

In [12]:
def add_filename(file, added):
    name, ext = os.path.splitext(file)
    return name + added + ext


def conf_matrix(y_true, y_pred, cls2dir, prop, title, annot=True, savepath=None):
    cm = confusion_matrix(y_true, y_pred)

    if prop:
        cm = (cm / len(y_true) * 100).astype(int)


    num = min(30, len(cm) // 2 + 1)
    
    labels = [cls2dir[i] for i in sorted(cls2dir.keys())]

    # fmt = 'f' if prop else 'd'
    # fmt = '.4g'
    fmt = 'd'

    plt.figure(figsize=(num, num))
    sns.heatmap(cm, annot=annot, fmt=fmt, cmap='nipy_spectral', xticklabels=labels, yticklabels=labels)
    plt.xlabel("Prediction")
    plt.ylabel("True")
    plt.xticks(rotation=45, ha='right')
    x = " probs" if prop else ""
    plt.title(f"Confusion Matrix{x}, {title}")

    if savepath:
        plt.savefig(savepath, bbox_inches='tight')


    if len(cm) > 30 and not prop:
        plt.clf()

        sns.heatmap(cm, annot=False, fmt=fmt, cmap='nipy_spectral', xticklabels=labels, yticklabels=labels)
        plt.xlabel("Prediction")
        plt.ylabel("True")
        plt.xticks(rotation=45, ha='right')
        plt.title(f"Confusion Matrix, {title}")

        if savepath:
            plt.savefig(add_filename(savepath, "_noAnnot"), bbox_inches='tight')


    plt.clf()
    plt.close()

    # plt.show()

## Model

In [13]:
class Pipeline(nn.Module):
    def __init__(self, model, cl):
        super().__init__()
        self.model = model
        self.cl = cl

    def forward(self, x):
        x = self.model(x)
        x = self.cl(x)
        return x


def test_pipeline(model, dataset, device, batch_size, collate_fn=None, type="siam", embs_model=None):

    print("--- test_pipeline:")

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    for batch in loader:
        if type == "siam":
            anchor, positive, negative = [d.to(device) for d in batch]
            anchor_output = model(anchor)
            positive_output = model(positive)
            negative_output = model(negative)

            print(f"anchor: {anchor.shape}\nmodel:   {anchor_output.shape}")
        else:

            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            if embs_model is None:
                raise TypeError("embs_model is None!!!")

            embs = embs_model(X_batch)
            output = model(embs) # torch.Size([BATCH, NUM_CLS])
            print(f"X_batch: {X_batch.shape}\ny_batch: {y_batch.shape}\nembs: {embs.shape}\nmodel:   {output.shape}")

        break

    print("--- test_pipeline end\n")


def create_model(model, num_not_freeze_layers, num_out_classes):
    # замена последнего слоя сети
    model.fc = nn.Linear(512, num_out_classes)

    num_freeze_layers = len(list(model.children())) - num_not_freeze_layers
    # заморозка слоев
    for i, layer in enumerate(model.children()):
        if i < num_freeze_layers:
            for param in layer.parameters():
                param.requires_grad = False
        else:
            for param in layer.parameters():
                param.requires_grad = True

    return model


def number_of_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def ToModel(model, device):
    def fun(tensor, model=model, device=device):
        return model(tensor[None, :].to(device))[0]
    return fun


def Min_size(size):
    def fun(tensor, size=size):
        c, h, w = tensor.shape
        new_h = size
        new_w = int(w * new_h / h)

        res = transforms.Resize((new_h, new_w))(tensor)
        return res

    return fun


def split_dataset(dataset, percent):
    train_size = int(len(dataset) * percent)
    val_size = len(dataset) - train_size

    train_dataset, test_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size]
    )

    len_tr, len_test = len(train_dataset), len(test_dataset)
    # print(f"split_dataset: Train: {len_tr} Test: {len_test} Total: {len_tr + len_test}")

    return train_dataset, test_dataset

## Load model

In [14]:
def load_model(model, path, device):
    print(f"load_model: {path}")
    model.load_state_dict(torch.load(path, map_location=device, weights_only=True))


def load_best_model(model, dir, device):
    file = [i for i in os.listdir(dir) if ".pickle" in i]

    if len(file) > 1:
        print(f"There are several pickles in {dir}: {file} !!!")

    file = file[0]

    path = os.path.join(dir, file)

    logs = load_logs(path)
    argmax_val_acc = logs["max_val_acc"]["arg"]
    load_model(model, os.path.join(dir, f"{argmax_val_acc} epoch.pt"), device)
    return model

In [15]:
import numpy as np
from sklearn.metrics import precision_score, accuracy_score, precision_recall_fscore_support

y_true = torch.tensor([0, 1, 2, 0, 1, 2, 3])
y_pred = torch.tensor([0, 2, 1, 0, 0, 1, 3])

precision_score(y_true, y_pred, average='macro')

0.41666666666666663

In [16]:
precision_recall_fscore_support(y_true, y_pred)

(array([0.66666667, 0.        , 0.        , 1.        ]),
 array([1., 0., 0., 1.]),
 array([0.8, 0. , 0. , 1. ]),
 array([2, 2, 2, 1]))

In [17]:
2/(3/2 + 1)

0.8

In [18]:
precision_score(y_true, y_pred, average=None)

array([0.66666667, 0.        , 0.        , 1.        ])

In [19]:
y_true, y_pred

(tensor([0, 1, 2, 0, 1, 2, 3]), tensor([0, 2, 1, 0, 0, 1, 3]))

In [20]:
y_true, y_pred

(tensor([0, 1, 2, 0, 1, 2, 3]), tensor([0, 2, 1, 0, 0, 1, 3]))

In [21]:
torch.mean((y_true == y_pred).float()).cpu().item()

0.4285714328289032

In [22]:
accuracy_score(y_true, y_pred)

0.42857142857142855

In [23]:
probs = torch.rand(100, 5)
probs = probs / probs.sum(dim=1)[:, None]
pred = torch.argmax(probs, dim=1)
true = torch.randint(0, probs.shape[1], size=(probs.shape[0], ))

cls2dir = {i : f"class {i}" for i in range(probs.shape[1])}

In [24]:
(pred == true).float().mean().item()

0.1899999976158142

In [25]:
accuracy_score(pred, true)

0.19

In [26]:
# hits_at_k(true, probs)

In [27]:
precision_recall_fscore_support(true, pred)

(array([0.18181818, 0.14285714, 0.22222222, 0.2       , 0.21052632]),
 array([0.19047619, 0.15789474, 0.21052632, 0.25      , 0.16      ]),
 array([0.18604651, 0.15      , 0.21621622, 0.22222222, 0.18181818]),
 array([21, 19, 19, 16, 25]))

In [28]:
# d = precision_recall_f1(true, probs, cls2dir)

# for dir, item in d.items():
#     print(dir)
#     recall = [f"k={k} : {val:.4f}" for k, val in item["Recall"].items()]
#     precision = [f"k={k} : {val:.4f}" for k, val in item["Precision"].items()]
#     f_score = [f"k={k} : {val:.4f}" for k, val in item["F1"].items()]
#     support = item["Support"]

#     print(f"Precision: {',   '.join(precision)}")
#     print(f"Recall:    {',   '.join(recall)}")
#     print(f"F1:        {',   '.join(f_score)}")
#     print(f"Support:   {support}")
#     print()

## Eval

In [29]:
def probs2topKpred(true, probs, topk):
    x_places = torch.argsort(torch.argsort(probs, dim=1, descending=True)) # в строке на i : место для i-го класса
    # x_places = torch.argsort(probs, dim=1, descending=True)
    places = x_places[torch.arange(x_places.shape[0]), true] # места для верных ответов

    # places = places < topk
    # return places.cpu() 
    true_pred = places < topk # вектор из True и False, True => в топ К кандидатах
    pred = torch.argmax(probs, dim=1)
    pred = torch.where(true_pred, true, pred)
    return pred # улучшенные предсказания


def hits_at_k(true, probs, k = [1, 2, 3, 4, 5]):
    d = {}
    for k_item in k:
        pred = probs2topKpred(true, probs, k_item)
        d[k_item] = accuracy_score(pred, true)
    return d


def precision_recall_f1(true, probs, cls2dir, k = [1, 2, 3, 4, 5]):
    d = {}

    d["Macro-Precision"] = {i : -1 for i in k}
    d["Macro-Recall"] = {i : -1 for i in k}
    d["Macro-F1"] = {i : -1 for i in k}

    d["dirs"] = list(cls2dir.values())

    for k_item in k:
        pred = probs2topKpred(true, probs, k_item) 

        precision, recall, f_score, support = precision_recall_fscore_support(true, pred, zero_division=np.nan)

        precision = np.where(precision==np.nan, -999, precision)
        recall = np.where(recall==np.nan, -999, recall)
        f_score = np.where(f_score==np.nan, -999, f_score)

        d["Macro-Precision"][k_item] = np.mean(precision)
        d["Macro-Recall"][k_item] = np.mean(recall)
        d["Macro-F1"][k_item] = np.mean(f_score)

        for cls in range(len(precision)):
            dir = cls2dir[cls]

            if dir not in d:
                d[dir] = {"Recall" : {i : -1 for i in k},
                          "Precision" : {i : -1 for i in k},
                          "F1" : {i : -1 for i in k},
                          "Support" : -1}
                
            d[dir]["Recall"][k_item] = recall[cls]
            d[dir]["Precision"][k_item] = precision[cls]
            d[dir]["F1"][k_item] = f_score[cls]
            d[dir]["Support"] = support[cls]
    return d


def evaluation_siam(model, dataloader, device):
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:

            anchor, positive, negative = [d.to(device) for d in batch]
            anchor_output = model(anchor)
            positive_output = model(positive)
            negative_output = model(negative)

            correct += (
                (torch.norm(anchor_output - positive_output, dim=1)
                < torch.norm(anchor_output - negative_output, dim=1)).sum()
                )
            total += anchor.size(0)

    return correct / total * 100


def reshape_epoches(x, n_epochs):
    """
    x ~ (n_epochs * n_elem, *)
    return x ~ (n_epochs, n_elem, *)
    """
    num, *s = x.shape

    x = x.reshape(n_epochs, num // n_epochs, *s)
    return x


def evaluation(model, dataloader, device, n_epochs=1, returnX=False, embs_model=None):

    # dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    pred = []
    prob = []
    y_items = []
    x_items = []

    with torch.no_grad():
        for _ in range(n_epochs):
            for batch in dataloader:

                X_batch, y_batch = batch
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                embs = embs_model(X_batch) if embs_model is not None else X_batch

                if returnX:
                    x_items.append(X_batch.cpu())
                    
                y_items.append(y_batch.cpu())

                output = model(embs)
                y_pred = torch.argmax(output, dim=1)
                pred.append(y_pred.cpu())

                y_prob = torch.softmax(output, dim=1)
                prob.append(y_prob.cpu())

    pred = torch.cat(pred).cpu()
    prob = torch.cat(prob).cpu()
    y_items = torch.cat(y_items).cpu()

    if returnX:
        # try:
        x_items = torch.cat(x_items).cpu()
        # except:
        #     pass

    d = {"pred" : pred,
        "prob" : prob,
        "y_items" : y_items}

    if returnX:
        d["x_items"] = x_items

    if n_epochs != 1:
        for key in d:
            d[key] = reshape_epoches(d[key], n_epochs)
            if key == "prob":
                d[key] = d[key].mean(dim=0)
            elif key == "y_items":
                d[key] = d[key].mode(dim=0).values

        d["pred"] = torch.argmax(d["prob"], dim=1)

    d["acc"] = torch.mean((d["pred"] == d["y_items"]).float()).item() * 100

    d["hits@k"] = hits_at_k(d["y_items"], d["prob"])
    # d["pred"], d["prob"], d["y_items"], d["x_items"], d["acc"], d["hits"]
    return d

## Ensemble

In [30]:
len("00,   k=5 : 1.0000")

18

In [31]:
# def hits2file(file, hits):
#     for k, acc in hits.items():
#          write2file(file, f'Hits@{k}: {acc:.6g}')
#     write2file(file, '')


def score2file(file, score, scorename):
    for k, acc in score.items():
         write2file(file, f'{scorename}@{k}: {acc:.6g}')
    write2file(file, '')


def precrec2file(file, d):
    
    score2file(file, d["Macro-Precision"], "Macro-Precision")
    score2file(file, d["Macro-Recall"], "Macro-Recall")
    score2file(file, d["Macro-F1"], "Macro-F1")

    for dir in d["dirs"]:
        item = d[dir]
        
        write2file(file, f"                   {dir}")
        recall = [f"k={k} : {val:.4f}" for k, val in item["Recall"].items()]
        precision = [f"k={k} : {val:.4f}" for k, val in item["Precision"].items()]
        f_score = [f"k={k} : {val:.4f}" for k, val in item["F1"].items()]
        support = item["Support"]

        write2file(file, f"Precision: {',   '.join(precision)}")
        write2file(file, f"Recall:    {',   '.join(recall)}")
        write2file(file, f"F1:        {',   '.join(f_score)}")
        write2file(file, f"Support:   {support}\n")

    write2file(file, f"\n")


def ensemble(model_all, dataset, batch_size, device, savedir, n_epochs):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    d = evaluation(model_all, dataloader, device=device, n_epochs=n_epochs)

    conf_matrix(d["y_items"], d["pred"], dataset.cls2dir, prop=False, title=f'acc: {d["acc"]:.6g}', savepath = os.path.join(savedir, f"conf{n_epochs}.pdf"))
    conf_matrix(d["y_items"], d["pred"], dataset.cls2dir, prop=True, title=f'acc: {d["acc"]:.6g}', savepath = os.path.join(savedir, f"conf{n_epochs}_prop.pdf"))
    fails = torch.where(d["y_items"] != d["pred"])[0]
    fails2file(dataset, d, fails, os.path.join(savedir, f"fails{n_epochs}.txt"))

    # Печать статистики
    file_info = os.path.join(savedir, "Main.txt")

    write2file(file_info, f' ---------------------------------  windows num = {n_epochs}  --------------------------------- \n') 
    score2file(file_info, d["hits@k"], "Hits")
    precrec2file(file_info, precision_recall_f1(d["y_items"], d["prob"], dataset.cls2dir))


def ensemble_strokes(model_all, dataset, batch_size, device, savedir, collate_fn):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    d = evaluation(model_all, dataloader, device=device)

    paths = [i[0] for i in dataset.dataset]
    paths2prob = {}
    for path, prob in zip(paths, d["prob"]):
        paths2prob[path] = prob

    images2strokes = defaultdict(list)
    for path in paths:
        # path ~ КККМ ОФ 7928:98 Агафонов # 2.png
        _, dir, picname = path.split("/")
        picname = os.path.splitext(picname)[0]
        picname, _ = picname.split(" # ")
        images2strokes[picname].append((path, dir))

    list_prob = []
    list_pred = []
    list_true = []
    list_images = []

    # print(images2strokes)
    # raise

    for image, paths_dirs in images2strokes.items():
        paths, dirs = zip(*paths_dirs)
        dir = dirs[0]

        list_probs = [paths2prob[path] for path in paths]
        list_probs = torch.vstack(list_probs)
        prob = torch.mean(list_probs, dim=0)
        pred = torch.argmax(prob).item()
        true = dataset.dir2cls[dir]

        list_images.append(image)
        list_prob.append(prob)
        list_pred.append(pred)
        list_true.append(true)


    list_pred = torch.tensor(list_pred)
    list_true = torch.tensor(list_true)
    list_prob = torch.vstack(list_prob)

    # print(list_prob.shape)

    d["y_items"], d["pred"], d["prob"] = list_true, list_pred, list_prob
    d["acc"]  = torch.mean((list_pred == list_true).float()).item() * 100
    d["hits@k"] = hits_at_k(d["y_items"], d["prob"])

    conf_matrix(d["y_items"], d["pred"], dataset.cls2dir, prop=False, title=f'acc: {d["acc"]:.6g}', savepath = os.path.join(savedir, f"conf.pdf"))
    conf_matrix(d["y_items"], d["pred"], dataset.cls2dir, prop=True, title=f'acc: {d["acc"]:.6g}', savepath = os.path.join(savedir, f"conf_prop.pdf"))
    fails = torch.where(d["y_items"] != d["pred"])[0]
    fails2file_strokes(list_images, dataset.cls2dir, d, fails, os.path.join(savedir, "fails.txt"))

    # Печать статистики
    file_info = os.path.join(savedir, "Main.txt")

    score2file(file_info, d["hits@k"], "Hits")
    precrec2file(file_info, precision_recall_f1(d["y_items"], d["prob"], dataset.cls2dir))


def fails2file_strokes(list_images, cls2dir, d, fails, file):
    if os.path.exists(file):
        !rm "{file}"
        print(f"Delete {file}")

    write2file(file, f'Fails num: {len(fails)} out of {len(list_images)}')
    write2file(file, f'accuracy: {d["acc"]:.6g}\n')

    hits = d["hits@k"]
    for k, acc in hits.items():
         write2file(file, f'Hits@{k}: {acc:.6g}')
    write2file(file, '')


    for num, i in enumerate(fails):
        write2file(file, f'{num}: {list_images[i]}')
        y_item = d["y_items"][i].item()
        pred = d["pred"][i].item()
        prob = d["prob"][i].numpy()

        write2file(file, f'True/Pred: {y_item} / {pred}, {cls2dir[y_item]} / {cls2dir[pred]}')

        args = np.argsort(prob)[::-1]
        place = np.where(args == y_item)[0][0] + 1

        write2file(file, f'Prob: {prob[pred]:.4f}, Place: {place}')

        args = args[:3]
        # print(f'Distr:{prob}')

        for k in args:
            write2file(file, f'{prob[k]:.4f} {cls2dir[k]}')

        write2file(file, "\n")


def fails2file(dataset, d, fails, file):
    if os.path.exists(file):
        !rm "{file}"
        print(f"Delete {file}")

    write2file(file, f'Fails num: {len(fails)} out of {len(dataset)}')
    write2file(file, f'accuracy: {d["acc"]:.6g}\n')

    hits = d["hits@k"]
    for k, acc in hits.items():
         write2file(file, f'Hits@{k}: {acc:.6g}')
    write2file(file, '')

    for num, i in enumerate(fails):
        write2file(file, f'{num}: {dataset.dataset[i][0]}')
        y_item = d["y_items"][i].item()
        pred = d["pred"][i].item()
        prob = d["prob"][i].numpy()

        write2file(file, f'True/Pred: {y_item} / {pred}, {dataset.cls2dir[y_item]} / {dataset.cls2dir[pred]}')

        args = np.argsort(prob)[::-1]
        place = np.where(args == y_item)[0][0] + 1

        write2file(file, f'Prob: {prob[pred]:.4f}, Place: {place}')

        args = args[:3]
        # print(f'Distr:{prob}')

        for k in args:
            write2file(file, f'{prob[k]:.4f} {dataset.cls2dir[k]}')

        write2file(file, "\n")

## Trainer

In [32]:
DEVICE=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {DEVICE}")

class Trainer:
    def global_print(self, text): # печать на экран и в savedir_logs_txt
        print(text)
        write2file(self.savedir_logs_txt, text)


    def message_fun(self, cur_time, epoch, train_loss, val_loss, train_acc, val_acc):
        return f"{cur_time} epoch: {epoch:4}, train/val loss = {train_loss:.5f} / {val_loss:.5f}, acc = {train_acc:7.3f} / {val_acc:7.3f}"


    def add_log(self, **args): # Добавить в self.logs словарь args
        for key in self.logs:
            if key not in args:
                self.global_print(f"{key} not in {list(args.keys())}")
            else: # key in args
                if type(self.logs[key]) is list:
                    self.logs[key].append(args[key])
                else:

                    if key[:3] == "min": # loss
                        if args[key]["value"] <= self.logs[key]["value"]:
                            self.logs[key] = args[key]
                    elif key[:3] == "max": # acc
                        if args[key]["value"] >= self.logs[key]["value"]:
                            self.logs[key] = args[key]


    def message_best_fun(self): # Информация о лучшем loss и accuracy
        mes  = ["train loss", "val loss  ", "train acc ", "val acc   "]
        keys = ["min_train_loss", "min_val_loss", "max_train_acc", "max_val_acc"]

        ans = []
        for key, m in zip(keys, mes):
            idx = self.logs[key]["arg"]
            value = self.logs[key]["value"]

            ans.append(f"Best {m} = {my_round(value, 10)} on {idx:4} epoch")

        return "\n".join(ans)


    def __init__(
            self,
            model,
            optimizer,
            criterion,
            type,
            name,
            device,
            embs_model=None,
            epoches_dir="logs",
            dir="info",
            experiment_id=None,
            early_stopping=10,
    ):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        self.stop = early_stopping
        self.embs_model = embs_model

        if type == "siam":
            self._train_step = self._train_step_siam
            self._eval_step = self._eval_step_siam
        else:
            type = "class"

            if self.embs_model is None:
                raise TypeError('type = "class", but embs_model = None')

            self.embs_model = self.embs_model.to(device)
            self.embs_model.eval()
            for p in self.embs_model.parameters():
                p.requires_grad = False

        # self.savedir = os.path.join(dir, f"{my_time()} {.name}") # название папки для сохранения
        os.makedirs(epoches_dir, exist_ok=True)
        os.makedirs(dir, exist_ok=True)

        if experiment_id is None:
            self.experiment_id = f"{my_time()} {name}"
        else:
            self.experiment_id = experiment_id

        self.savedir_epochs = os.path.join(epoches_dir, self.experiment_id)
        self.savedir_info = os.path.join(dir, self.experiment_id)
        self.savedir_logs_txt = os.path.join(self.savedir_info, f"logs {type}.txt")

        if experiment_id is None:
            os.makedirs(self.savedir_epochs, exist_ok=False)
            os.makedirs(self.savedir_info, exist_ok=False)
            self.global_print(f"Make {self.savedir_epochs}")
            self.global_print(f"Make {self.savedir_info}")


        self.savedir_epochs = os.path.join(epoches_dir, self.experiment_id, type)
        os.makedirs(self.savedir_epochs, exist_ok=False)
        self.savedir_logs_dict = os.path.join(self.savedir_epochs, "logs.pickle")
        self.savedir_epochs_fun = lambda i : f"{self.savedir_epochs}/{i} epoch.pt"
        self.not_decrease = "Val losses doesn`t decrease!"

        self.tqdm_disable = False

        self._n_epoch = 0

        self.logs = {
            "time":  [],
            "epoch": [],
            "train_loss":   [],
            "val_loss":     [],
            "train_acc":    [],
            "val_acc":      [],
            "min_train_loss":   {"arg" : None, "value" : float("inf")},
            "min_val_loss":     {"arg" : None, "value" : float("inf")},
            "max_train_acc":    {"arg" : None, "value" : -1},
            "max_val_acc":      {"arg" : None, "value" : -1},
        }




    def train(self, train_loader, val_loader, n_epochs):

        self.global_print("\nTraining start")
        start_time = time()

        val_losses = self.logs["val_loss"]

        for epoch in tqdm_n(range(n_epochs), disable=False):

            train_loss, train_acc = self._train_step(train_loader)
            val_loss, val_acc = self._eval_step(val_loader)

            if not self.tqdm_disable:
                self.tqdm_disable = True

            cur_time = my_timediff(start_time)

            self.global_print(self.message_fun(cur_time, self._n_epoch, train_loss, val_loss, train_acc, val_acc))

            args = {
                "time": cur_time,
                "epoch": self._n_epoch,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "train_acc": train_acc,
                "val_acc": val_acc,
                "min_train_loss":   {"arg" : self._n_epoch, "value" : train_loss},
                "min_val_loss":     {"arg" : self._n_epoch, "value" : val_loss},
                "max_train_acc":    {"arg" : self._n_epoch, "value" : train_acc},
                "max_val_acc":      {"arg" : self._n_epoch, "value" : val_acc},
            }
            self.add_log(**args)

            torch.save(self.model.state_dict(), self.savedir_epochs_fun(epoch))
            save_logs(self.logs, self.savedir_logs_dict)

            if len(val_losses) > self.stop and min(val_losses[-self.stop:]) > val_losses[-self.stop-1]:
                self.global_print(self.not_decrease)

            self._n_epoch += 1

        self.global_print("")
        self.global_print(self.message_best_fun())

        self.global_print("Training completed\n")
        return self.savedir_epochs, self.savedir_info, self.experiment_id


    def _train_step(self, dataloader):
        self.model.train()

        total_loss = 0
        correct = 0
        total = 0

        for batch in tqdm_n(dataloader, disable=self.tqdm_disable):

            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)
            embs = self.embs_model(X_batch)

            output = self.model(embs) # torch.Size([BATCH, NUM_CLS])
            loss = self.criterion(output, y_batch)

            y_pred = torch.argmax(output, dim=1)
            correct += torch.sum(y_pred == y_batch)
            total += len(y_batch)

            total_loss += loss
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

        return total_loss.item() / len(dataloader), correct.item() / total * 100


    def _eval_step(self, dataloader):
        self.model.eval()

        total_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in tqdm_n(dataloader, disable=self.tqdm_disable):

                X_batch, y_batch = batch
                X_batch, y_batch = X_batch.to(self.device), y_batch.to(self.device)
                embs = self.embs_model(X_batch)

                output = self.model(embs)
                loss = self.criterion(output, y_batch)

                y_pred = torch.argmax(output, dim=1)
                correct += torch.sum(y_pred == y_batch)
                total += len(y_batch)

                total_loss += loss

        return total_loss.item() / len(dataloader), correct.item() / total * 100


    def _train_step_siam(self, dataloader):
        total_loss = 0
        correct = 0
        total = 0

        for batch in tqdm_n(dataloader, disable=self.tqdm_disable):

            anchor, positive, negative = [d.to(self.device) for d in batch]
            anchor_output = self.model(anchor)
            positive_output = self.model(positive)
            negative_output = self.model(negative)

            loss = self.criterion(anchor_output, positive_output, negative_output)

            correct += (
                (torch.norm(anchor_output - positive_output, dim=1)
                 < torch.norm(anchor_output - negative_output, dim=1)).sum()
                 )
            total += anchor.size(0)

            total_loss += loss

            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

        return total_loss.item() / len(dataloader), correct.item() / total * 100


    def _eval_step_siam(self, dataloader):
        total_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in tqdm_n(dataloader, disable=self.tqdm_disable):

                anchor, positive, negative = [d.to(self.device) for d in batch]
                anchor_output = self.model(anchor)
                positive_output = self.model(positive)
                negative_output = self.model(negative)

                loss = self.criterion(anchor_output, positive_output, negative_output)

                correct += (
                    (torch.norm(anchor_output - positive_output, dim=1)
                    < torch.norm(anchor_output - negative_output, dim=1)).sum()
                    )
                total += anchor.size(0)

                total_loss += loss

        return total_loss.item() / len(dataloader), correct.item() / total * 100

Device: cuda


## Strokes

In [33]:
PAD_ID = -1

def pad_collate(batch):
    # BATCH_SIZE * C * EMB_SIZE x SEQ_LEN
    list_tensors, list_targets = zip(*batch)
    # print(len(list_tersors), len(list_targets))
    # print(list_tersors[0].shape)
    ans = [None] * len(list_tensors)
    # tokens_len = [None] * len(list_tersors)
    # masks = [None] * len(list_tersors)

    for i in range(len(list_tensors)):
        # tokens_len[i] = list_tensors[i].shape[-1] # SEQ_LEN
        ans[i] = list_tensors[i].permute(2, 0, 1) # C x EMB_SIZE x SEQ_LEN -> SEQ_LEN x C x EMB_SIZE

    # BATCH_SIZE * SEQ_LEN * C * EMB_SIZE ->
    # -> BATCH_SIZE x C x EMB_SIZE x SEQ_LEN
    return pad_sequence(ans, batch_first=True, padding_value=PAD_ID).permute(0, 2, 3, 1), torch.tensor(list_targets)


def pad(ans):
    return pad_sequence(ans, batch_first=True, padding_value=PAD_ID).permute(0, 2, 3, 1)

def pad_collate_siam(batch):
    # BATCH_SIZE * C * EMB_SIZE x SEQ_LEN
    list_tensors1, list_tensors2, list_tensors3 = zip(*batch)
    batch_size = len(list_tensors1)
    list_tensors = list_tensors1 + list_tensors2 + list_tensors3

    ans = [None] * len(list_tensors)
    # tokens_len = [None] * len(list_tersors)
    # masks = [None] * len(list_tersors)

    for i in range(len(list_tensors)):
        # tokens_len[i] = list_tensors[i].shape[-1] # SEQ_LEN
        ans[i] = list_tensors[i].permute(2, 0, 1) # C x EMB_SIZE x SEQ_LEN -> SEQ_LEN x C x EMB_SIZE

    # BATCH_SIZE * SEQ_LEN * C * EMB_SIZE ->
    # -> BATCH_SIZE x C x EMB_SIZE x SEQ_LEN

    ans1, ans2, ans3 = ans[:batch_size], ans[batch_size : 2 * batch_size], ans[2 * batch_size:]
    return pad(ans1), pad(ans2), pad(ans3)


class NetworkStrokes(nn.Module):
    def __init__(self, in_channels=3, out_channels=10, emb_size=18, hid_size=128, output_emb = 1000, num_layers=1):
        super().__init__()
        self.CNN = nn.Sequential(
                     nn.Conv2d(in_channels=in_channels, out_channels=3, kernel_size=3),
                     nn.BatchNorm2d(3),
                     nn.ReLU(),
                     nn.MaxPool2d(kernel_size=2, stride=(2, 3)),

                     nn.Conv2d(in_channels=3, out_channels=5, kernel_size=3),
                     nn.BatchNorm2d(5),
                     nn.ReLU(),
                     nn.MaxPool2d(kernel_size=2, stride=(1, 3)),

                     nn.Conv2d(in_channels=5, out_channels=out_channels, kernel_size=3),
                     nn.BatchNorm2d(10),
                     nn.ReLU(),
                     nn.MaxPool2d(kernel_size=2, stride=(1, 2)),
                     )

        self.linear = nn.Linear(out_channels, 1)

        self.LSTM = nn.LSTM(input_size=emb_size, # emb_size зависит от CNN
                            hidden_size=hid_size,
                            num_layers=num_layers,
                            batch_first=True)

        self.classifier = nn.Sequential(nn.Linear(hid_size, output_emb))

    def forward(self, x):
        # x ~ BATCH_SIZE x C x EMB_SIZE x SEQ_LEN
        # tokens_lens ~ BATCH_SIZE
        mask = (x != PAD_ID).int()
        # print(x.shape, mask.shape)
        # print(1, x.shape)
        x = self.CNN(x) # BATCH_SIZE x out_channels x EMB_SIZE_new x SEQ_LEN_new
        # print(2, x.shape)
        x = x.permute(0, 2, 3, 1) # BATCH_SIZE x EMB_SIZE_new x SEQ_LEN_new x out_channels
        x = self.linear(x).squeeze(dim=-1) # BATCH_SIZE x EMB_SIZE_new x SEQ_LEN_new
        x = x.permute(0, 2, 1) # BATCH_SIZE x SEQ_LEN_new x EMB_SIZE_new


        mask = mask[:, 0, 0, ::18] # BATCH_SIZE x SEQ_LEN_new
        mask = mask[:, :x.shape[1]]

        # print(3, x.shape)
        x, _ = self.LSTM(x) # BATCH_SIZE x SEQ_LEN_new x HIDDEN_SIZE

        # print(x.shape, mask.shape)
        x = x * mask[..., None] # BATCH_SIZE x SEQ_LEN_new x HIDDEN_SIZE

        sum = x.sum(dim = 1) # BATCH_SIZE x HIDDEN_SIZE
        col = mask.sum(dim = 1) # BATCH_SIZE

        x = sum / col[..., None]
        # x = x.mean(dim = 1)
        # last_hid_idxs = (tokens_lens / 18).to(int) - 1
        # last_hid_idxs = (tokens_lens / 18).to(int) - 5
        # x = x[torch.arange(len(tokens_lens)), last_hid_idxs - 1, :] # BATCH_SIZE x HIDDEN_SIZE
        # BATCH_SIZE x HIDDEN_SIZE
        return self.classifier(x)

## Experiment code (window)

In [34]:
def experiment(config):
    print(f'Device: {config["device"]}')

    PAD_ID = -1
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    model = create_model(model,
                            num_not_freeze_layers=config["num_not_freeze_layers"],
                            num_out_classes=config["siam_emb_dim"]).to(config["device"])

    print(f"Siam model parameters number: {number_of_parameters(model)}")

    transform = transforms.Compose(
    [
            transforms.RandomCrop(300),
            transforms.ToTensor(),
    ])

    if not config["balanced"]:
        dataset = My_TripletFolder(config["folder"], transform=transform, topk=config["dataset_topk"])
        print(f"\nDataset length: {len(dataset)}")
        for key, value in dataset.dir2count.items():
            print(f"{key:50}   :   {value}")

        train_dataset, test_dataset = split_dataset(dataset, percent=config["split_percent"])
        print(f"train dataset: {len(train_dataset)}, test dataset: {len(test_dataset)}\n")
    else:
        train_dataset, test_dataset = My_Balanced_split_dataset(config["folder"], transform=transform, topk=config["dataset_topk"],
                                                                percent=config["split_percent"], type="siam")

        print("\ntrain + test dataset")
        for key, value in train_dataset.dir2count.items():
            test_value = test_dataset.dir2count[key]
            print(f"{key:50}   :   {value + test_value} = {value} + {test_value}")

        print(f"train dataset: {len(train_dataset)}, test dataset: {len(test_dataset)}\n")

    test_pipeline(model, train_dataset, device=config["device"], batch_size=config["batch_size"], type = "siam")

    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    criterion = torch.nn.TripletMarginLoss(margin=9, p=2)

    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

    if "trained" not in config:
        x = "_balanced" if config["balanced"] else ""
        trainer = Trainer(model, optimizer, criterion, type = "siam", name = f'siam_{config["type"]}{config["dataset_topk"]}{x}', device=config["device"])
        savedir_epochs, savedir_info, experiment_id = trainer.train(train_loader, val_loader, n_epochs = config["siam_epochs"])
        # end siam train
    else:
        name = f'siam_{config["type"]}{config["dataset_topk"]}{"_balanced" if config["balanced"] else ""}'

        pos_name = [i for i in os.listdir("logs") if name == i.split()[-1]]

        if len(pos_name) == 1:
            pos_name = pos_name[0]
            print(f"Find {pos_name}")
        elif len(pos_name) == 0:
            raise TypeError(f"Didn't find {name} !!!")
        else:
            raise TypeError(f"Found: {pos_name} !!!")
        
        savedir_epochs = f"logs/{pos_name}/siam"
        savedir_info = f"info/{pos_name}"
        experiment_id = pos_name

    model = load_best_model(model, dir=savedir_epochs, device=config["device"])
    print(f'train/val acc : {evaluation_siam(model, train_loader, device=config["device"])} / {evaluation_siam(model, val_loader, device=config["device"])}')

    model.eval()
    for p in model.parameters():
        p.requires_grad = False
    print(f"Siam model parameters number: {number_of_parameters(model)}")

    # start cl train
    cl_model = nn.Sequential(nn.Linear(config["siam_emb_dim"], 512), nn.ReLU(), nn.Linear(512, config["dataset_topk"])).to(config["device"])
    print(f"Class model parameters number: {number_of_parameters(cl_model)}")

    if not config["balanced"]:
        dataset = My_Folder(config["folder"], transform, topk=config["dataset_topk"])
        print(f"Dataset length: {len(dataset)}")
        train_dataset, test_dataset = split_dataset(dataset, percent=config["split_percent"])
        print(f"train dataset: {len(train_dataset)}, test dataset: {len(test_dataset)}\n")
    else:
        train_dataset, test_dataset = My_Balanced_split_dataset(config["folder"], transform=transform, topk=config["dataset_topk"],
                                                                percent=config["split_percent"], type="class")
        print(f"train dataset: {len(train_dataset)}, test dataset: {len(test_dataset)}\n")

    test_pipeline(cl_model, train_dataset, device=config["device"], batch_size=config["batch_size"], type = "class", embs_model=model)

    optimizer = torch.optim.Adam(cl_model.parameters(), lr=3e-4)
    criterion = torch.nn.CrossEntropyLoss()

    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

    if "trained" not in config:
        trainer = Trainer(cl_model, optimizer, criterion, type = "class", name = f'cl_{config["type"]}{config["dataset_topk"]}{x}', device=config["device"],
                        experiment_id=experiment_id, embs_model=model)
        savedir_epochs, savedir_info, experiment_id = trainer.train(train_loader, val_loader, n_epochs = config["cl_epochs"])
    else:
        savedir_epochs = f"logs/{pos_name}/class"

    cl_model = load_best_model(cl_model, dir=savedir_epochs, device=config["device"])
    d1 = evaluation(cl_model, train_loader, device=config["device"], embs_model=model)
    d2 = evaluation(cl_model, val_loader, device=config["device"], embs_model=model)

    print(f'train/val acc : {d1["acc"]} / {d2["acc"]}\nhits@k: {d1["hits@k"]}\n    /   {d2["hits@k"]}')

    cl_model.eval()
    for p in cl_model.parameters():
        p.requires_grad = False
    print(f"Class model parameters number: {number_of_parameters(cl_model)}")
    # end cl train

    # ensemble
    model_all = Pipeline(model, cl_model).to(config["device"])
    print(f"Model parameters number: {number_of_parameters(model_all)}")

    dataset = My_Folder(config["folder"], transform, topk=config["dataset_topk"])


    file = os.path.join(savedir_info, "Main.txt")
    if os.path.exists(file):
        !rm "{file}"
        print(f"Delete {file}")

    ensemble(model_all, dataset, config["batch_size"], config["device"], savedir_info, n_epochs=1) 
    ensemble(model_all, dataset, config["batch_size"], config["device"], savedir_info, n_epochs=5)
    ensemble(model_all, dataset, config["batch_size"], config["device"], savedir_info, n_epochs=10)
    ensemble(model_all, dataset, config["batch_size"], config["device"], savedir_info, n_epochs=20)

## Experiment code (strokes)

In [35]:
def experiment_strokes(config):
    print(f'Device: {config["device"]}')

    if "model" not in config:
        PAD_ID = -1
        model = NetworkStrokes(output_emb=config["siam_emb_dim"]).to(config["device"])

        print(f"Siam model parameters number: {number_of_parameters(model)}")
        
    elif config["model"] == "ResNet":
        PAD_ID = 0
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model = create_model(model,
                                num_not_freeze_layers=config["num_not_freeze_layers"],
                                num_out_classes=config["siam_emb_dim"]).to(config["device"])

        print(f"Siam ResNet model parameters number: {number_of_parameters(model)}")
    elif config["model"] == "model2":
        PAD_ID = 0
        model = NetworkStrokes2(output_emb=config["siam_emb_dim"]).to(config["device"])
        print(f"Siam model parameters number: {number_of_parameters(model)}")
        

    transform = transforms.Compose(
    [
            transforms.ToTensor(),
            Min_size(50),
    ])

    if not config["balanced"]:
        dataset = My_TripletFolder(config["folder"], transform=transform, topk=config["dataset_topk"])
        print(f"\nDataset length: {len(dataset)}")
        for key, value in dataset.dir2count.items():
            print(f"{key:50}   :   {value}")

        train_dataset, test_dataset = split_dataset(dataset, percent=config["split_percent"])
        print(f"train dataset: {len(train_dataset)}, test dataset: {len(test_dataset)}\n")
    else:
        train_dataset, test_dataset = My_Balanced_split_dataset(config["folder"], transform=transform, topk=config["dataset_topk"],
                                                                percent=config["split_percent"], type="siam")

        print("\ntrain + test dataset")
        for key, value in train_dataset.dir2count.items():
            test_value = test_dataset.dir2count[key]
            print(f"{key:50}   :   {value + test_value} = {value} + {test_value}")

        print(f"train dataset: {len(train_dataset)}, test dataset: {len(test_dataset)}\n")

    test_pipeline(model, train_dataset, device=config["device"], batch_size=config["batch_size"],
                  collate_fn = pad_collate_siam, type = "siam")

    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    criterion = torch.nn.TripletMarginLoss(margin=9, p=2)

    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, collate_fn=pad_collate_siam)
    val_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, collate_fn=pad_collate_siam)

    if "trained" not in config:
        x = '_balanced' if config["balanced"] else ""
        trainer = Trainer(model, optimizer, criterion, type = "siam",
                        name = f'siam_{config["type"]}{config["dataset_topk"]}{x}', device=config["device"])
        savedir_epochs, savedir_info, experiment_id = trainer.train(train_loader, val_loader, n_epochs = config["siam_epochs"])
        # end siam train
    else:
        name = f'siam_{config["type"]}{config["dataset_topk"]}{"_balanced" if config["balanced"] else ""}'

        pos_name = [i for i in os.listdir("logs") if name == i.split()[-1]]

        if len(pos_name) == 1:
            pos_name = pos_name[0]
            print(f"Find {pos_name}")
        elif len(pos_name) == 0:
            raise TypeError(f"Didn't find {name} !!!")
        else:
            raise TypeError(f"Found: {pos_name} !!!")
        
        savedir_epochs = f"logs/{pos_name}/siam"
        savedir_info = f"info/{pos_name}"
        experiment_id = pos_name

    model = load_best_model(model, dir=savedir_epochs, device=config["device"])
    print(f'train/val acc : {evaluation_siam(model, train_loader, device=config["device"])} / {evaluation_siam(model, val_loader, device=config["device"])}')

    model.eval()
    for p in model.parameters():
        p.requires_grad = False
    print(f"Siam model parameters number: {number_of_parameters(model)}")

    # start cl train
    cl_model = nn.Sequential(nn.Linear(config["siam_emb_dim"], 512), nn.ReLU(), nn.Linear(512, config["dataset_topk"])).to(DEVICE)
    print(f"Class model parameters number: {number_of_parameters(cl_model)}")

    if not config["balanced"]:
        dataset = My_Folder(config["folder"], transform, topk=config["dataset_topk"])
        print(f"Dataset length: {len(dataset)}")
        train_dataset, test_dataset = split_dataset(dataset, percent=config["split_percent"])
        print(f"train dataset: {len(train_dataset)}, test dataset: {len(test_dataset)}\n")
    else:
        train_dataset, test_dataset = My_Balanced_split_dataset(config["folder"], transform=transform, topk=config["dataset_topk"],
                                                                percent=config["split_percent"], type="class")
        print(f"train dataset: {len(train_dataset)}, test dataset: {len(test_dataset)}\n")

    test_pipeline(cl_model, train_dataset, device=config["device"], batch_size=config["batch_size"],
                  collate_fn=pad_collate,
                  type = "class", embs_model=model)

    optimizer = torch.optim.Adam(cl_model.parameters(), lr=3e-4)
    criterion = torch.nn.CrossEntropyLoss()

    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, collate_fn=pad_collate)
    val_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, collate_fn=pad_collate)

    if "trained" not in config:
        trainer = Trainer(cl_model, optimizer, criterion, type = "class", name = f'cl_{config["type"]}{config["dataset_topk"]}{x}', device=config["device"],
                        experiment_id=experiment_id, embs_model=model)
        savedir_epochs, savedir_info, experiment_id = trainer.train(train_loader, val_loader, n_epochs = config["cl_epochs"])
    else:
        savedir_epochs = f"logs/{pos_name}/class"

    cl_model = load_best_model(cl_model, dir=savedir_epochs, device=config["device"])
    d1 = evaluation(cl_model, train_loader, device=config["device"], embs_model=model)
    d2 = evaluation(cl_model, val_loader, device=config["device"], embs_model=model)

    print(f'train/val acc : {d1["acc"]} / {d2["acc"]}\nhits@k: {d1["hits@k"]}\n    /   {d2["hits@k"]}')

    cl_model.eval()
    for p in cl_model.parameters():
        p.requires_grad = False
    print(f"Class model parameters number: {number_of_parameters(cl_model)}")
    # end cl train

    # ensemble
    model_all = Pipeline(model, cl_model).to(config["device"])
    print(f"Model parameters number: {number_of_parameters(model_all)}")

    dataset = My_Folder(config["folder"], transform, topk=config["dataset_topk"])

    file = os.path.join(savedir_info, "Main.txt")
    if os.path.exists(file):
        !rm "{file}"
        print(f"Delete {file}")

    ensemble_strokes(model_all, dataset, config["batch_size"], config["device"], savedir_info, collate_fn=pad_collate)

# Experiments

## Windows

In [60]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(DEVICE)
number_of_parameters(model)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for i in model.children():
    print(number_of_parameters(i))

9408
128
0
0
147968
525568
2099712
8393728
0
513000


### 3 class

#### Unbalanced 

In [33]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

Dataset length: 177
Фарафонтова Таисия Михайловна   :   67
Потанина Александра Викторовна   :   62
Ивановский Алексей Осипович   :   48

train dataset: 141, test dataset: 36
--- test_pipeline:
anchor: torch.Size([64, 3, 300, 300])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-03 18:47:02 siam_window3
Make info/2025-05-03 18:47:02 siam_window3

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

00:00:06 epoch:    0, train/val loss = 7.56286 / 6.97037, acc =  65.957 /  75.000
00:00:10 epoch:    1, train/val loss = 7.45991 / 7.03570, acc =  69.504 /  66.667
00:00:15 epoch:    2, train/val loss = 6.75322 / 6.00760, acc =  73.759 /  80.556
00:00:20 epoch:    3, train/val loss = 5.96904 / 5.68805, acc =  75.177 /  83.333
00:00:24 epoch:    4, train/val loss = 4.94436 / 5.77309, acc =  75.887 /  77.778
00:00:29 epoch:    5, train/val loss = 5.70128 / 4.93131, acc =  76.596 /  88.889
00:00:34 epoch:    6, train/val loss = 5.68105 / 4.91775, acc =  84.397 /  72.222
00:00:38 epoch:    7, train/val loss = 5.76893 / 4.37771, acc =  83.688 /  80.556
00:00:43 epoch:    8, train/val loss = 3.94024 / 3.69385, acc =  82.270 /  86.111
00:00:48 epoch:    9, train/val loss = 4.82934 / 4.12629, acc =  76.596 /  75.000
00:00:52 epoch:   10, train/val loss = 4.39221 / 4.34900, acc =  82.270 /  86.111
00:00:57 epoch:   11, train/val loss = 4.75634 / 3.86181, acc =  80.851 /  83.333
00:01:02 epoch: 

...

train/val acc : 87.9432601928711 / 77.77777862548828
Siam model parameters number: 0
Class model parameters number: 514051
Dataset length: 177
train dataset: 141, test dataset: 36

--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

00:00:01 epoch:    0, train/val loss = 0.86552 / 0.48838, acc =  56.738 /  91.667
00:00:03 epoch:    1, train/val loss = 0.57493 / 0.31144, acc =  82.270 /  91.667
00:00:04 epoch:    2, train/val loss = 0.55080 / 0.30009, acc =  85.816 /  97.222
00:00:05 epoch:    3, train/val loss = 0.47601 / 0.22097, acc =  86.525 /  94.444
00:00:07 epoch:    4, train/val loss = 0.39569 / 0.17275, acc =  85.816 /  94.444
00:00:08 epoch:    5, train/val loss = 0.31685 / 0.25340, acc =  86.525 /  94.444
00:00:10 epoch:    6, train/val loss = 0.42119 / 0.26654, acc =  89.362 /  88.889
00:00:11 epoch:    7, train/val loss = 0.40169 / 0.19016, acc =  85.106 /  91.667
00:00:13 epoch:    8, train/val loss = 0.35416 / 0.24434, acc =  83.688 /  91.667
00:00:14 epoch:    9, train/val loss = 0.24061 / 0.17690, acc =  87.234 /  94.444
00:00:15 epoch:   10, train/val loss = 0.27391 / 0.17558, acc =  87.234 /  94.444
00:00:17 epoch:   11, train/val loss = 0.29328 / 0.30363, acc =  87.943 /  94.444
00:00:18 epoch: 

In [36]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

Dataset length: 177
Фарафонтова Таисия Михайловна                        :   67
Потанина Александра Викторовна                       :   62
Ивановский Алексей Осипович                          :   48
train dataset: 141, test dataset: 36

--- test_pipeline:
anchor: torch.Size([64, 3, 300, 300])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-03 18:47:02 siam_window3
load_model: logs/2025-05-03 18:47:02 siam_window3/siam/28 epoch.pt
train/val acc : 87.23403930664062 / 86.1111068725586
Siam model parameters number: 0
Class model parameters number: 514051
Dataset length: 177
train dataset: 141, test dataset: 36

--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

load_model: logs/2025-05-03 18:47:02 siam_window3/class/27 epoch.pt
train/val acc : 88.65247964859009 / 86.11111044883728
hits@k: {1: 0.886524822695

#### Balanced

In [34]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_bin",
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

train dataset
Фарафонтова Таисия Михайловна   :   53
Потанина Александра Викторовна   :   49
Ивановский Алексей Осипович   :   38

test dataset
Фарафонтова Таисия Михайловна   :   14
Потанина Александра Викторовна   :   13
Ивановский Алексей Осипович   :   10

train dataset: 140, test dataset: 37
--- test_pipeline:
anchor: torch.Size([64, 3, 300, 300])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-03 18:50:18 siam_window3_balanced
Make info/2025-05-03 18:50:18 siam_window3_balanced

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

00:00:04 epoch:    0, train/val loss = 7.29910 / 7.56591, acc =  73.571 /  67.568
00:00:09 epoch:    1, train/val loss = 7.28062 / 7.19615, acc =  72.857 /  70.270
00:00:13 epoch:    2, train/val loss = 6.93006 / 7.04489, acc =  72.143 /  72.973
00:00:18 epoch:    3, train/val loss = 6.71951 / 6.41104, acc =  78.571 /  75.676
00:00:23 epoch:    4, train/val loss = 4.92076 / 5.82745, acc =  82.857 /  83.784
00:00:28 epoch:    5, train/val loss = 5.71072 / 7.36768, acc =  82.857 /  62.162
00:00:33 epoch:    6, train/val loss = 4.66059 / 4.28078, acc =  82.857 /  78.378
00:00:37 epoch:    7, train/val loss = 4.02588 / 5.65624, acc =  84.286 /  75.676
00:00:42 epoch:    8, train/val loss = 5.84417 / 5.42514, acc =  78.571 /  75.676
00:00:47 epoch:    9, train/val loss = 4.84418 / 5.65913, acc =  75.000 /  70.270
00:00:51 epoch:   10, train/val loss = 5.58320 / 5.40944, acc =  79.286 /  81.081
00:00:56 epoch:   11, train/val loss = 4.05210 / 4.68103, acc =  85.714 /  83.784
00:01:01 epoch: 

...

train/val acc : 82.14286041259766 / 83.7837905883789
Siam model parameters number: 0
Class model parameters number: 514051
train dataset: 140, test dataset: 37
--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

00:00:01 epoch:    0, train/val loss = 0.80035 / 0.57228, acc =  48.571 /  75.676
00:00:02 epoch:    1, train/val loss = 0.58630 / 0.53043, acc =  80.000 /  81.081
00:00:04 epoch:    2, train/val loss = 0.37692 / 0.34379, acc =  85.714 /  89.189
00:00:05 epoch:    3, train/val loss = 0.31471 / 0.39777, acc =  87.857 /  83.784
00:00:06 epoch:    4, train/val loss = 0.35195 / 0.22470, acc =  85.714 /  91.892
00:00:08 epoch:    5, train/val loss = 0.26294 / 0.37914, acc =  90.000 /  89.189
00:00:10 epoch:    6, train/val loss = 0.28668 / 0.34780, acc =  90.000 /  89.189
00:00:11 epoch:    7, train/val loss = 0.40649 / 0.33628, acc =  87.857 /  83.784
00:00:12 epoch:    8, train/val loss = 0.27935 / 0.25191, acc =  87.857 /  94.595
00:00:14 epoch:    9, train/val loss = 0.27501 / 0.19570, acc =  85.000 /  91.892
00:00:15 epoch:   10, train/val loss = 0.30179 / 0.29550, acc =  88.571 /  89.189
00:00:17 epoch:   11, train/val loss = 0.40049 / 0.23063, acc =  83.571 /  86.486
00:00:18 epoch: 

In [37]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_bin",
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

train + test dataset
Фарафонтова Таисия Михайловна                        :   67 = 53 + 14
Потанина Александра Викторовна                       :   62 = 49 + 13
Ивановский Алексей Осипович                          :   48 = 38 + 10
train dataset: 140, test dataset: 37

--- test_pipeline:
anchor: torch.Size([64, 3, 300, 300])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-03 18:50:18 siam_window3_balanced
load_model: logs/2025-05-03 18:50:18 siam_window3_balanced/siam/27 epoch.pt
train/val acc : 90.0 / 89.1891860961914
Siam model parameters number: 0
Class model parameters number: 514051
train dataset: 140, test dataset: 37

--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

load_model: logs/2025-05-03 18:50:18 siam_window3_balanced/class/25 epoch.pt
train/val acc : 92.85714030265808 / 91.89189076423645
h

### 10 class

#### Unbalanced 

In [35]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

Dataset length: 410
Фарафонтова Таисия Михайловна   :   67
Потанина Александра Викторовна   :   62
Ивановский Алексей Осипович   :   48
Крутовский Владимир Михайлович   :   46
Свентицкая Мария Хрисанфовна   :   35
Вагнер (Дзвонкевич) Екатерина Николаевна   :   32
Антонова Валентина Константиновна   :   31
Проскурякова Е. Павловна   :   30
Семидалов Вениамин Иванович   :   30
Мендельсон Николай Михайлович   :   29

train dataset: 328, test dataset: 82
--- test_pipeline:
anchor: torch.Size([64, 3, 300, 300])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-03 18:53:34 siam_window10
Make info/2025-05-03 18:53:34 siam_window10

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

00:00:12 epoch:    0, train/val loss = 8.17637 / 8.24192, acc =  67.073 /  60.976
00:00:23 epoch:    1, train/val loss = 7.65136 / 7.49222, acc =  70.732 /  68.293
00:00:33 epoch:    2, train/val loss = 7.31211 / 6.89506, acc =  71.646 /  70.732
00:00:44 epoch:    3, train/val loss = 7.38614 / 6.37201, acc =  71.341 /  74.390
00:00:54 epoch:    4, train/val loss = 6.90167 / 7.48506, acc =  68.598 /  70.732
00:01:05 epoch:    5, train/val loss = 6.31947 / 6.26389, acc =  73.780 /  71.951
00:01:15 epoch:    6, train/val loss = 6.16267 / 6.79027, acc =  74.390 /  75.610
00:01:25 epoch:    7, train/val loss = 6.50856 / 6.30725, acc =  71.951 /  75.610
00:01:36 epoch:    8, train/val loss = 6.44193 / 5.92933, acc =  73.476 /  68.293
00:01:46 epoch:    9, train/val loss = 5.89526 / 5.95225, acc =  72.561 /  69.512
00:01:57 epoch:   10, train/val loss = 6.27998 / 6.13720, acc =  72.866 /  74.390
00:02:07 epoch:   11, train/val loss = 5.86525 / 5.80790, acc =  74.695 /  74.390
00:02:17 epoch: 

...

train/val acc : 78.65853118896484 / 74.3902359008789
Siam model parameters number: 0
Class model parameters number: 517642
Dataset length: 410
train dataset: 328, test dataset: 82

--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 10])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

00:00:02 epoch:    0, train/val loss = 1.90537 / 1.82795, acc =  36.890 /  35.366
00:00:06 epoch:    1, train/val loss = 1.60031 / 1.60819, acc =  44.512 /  42.683
00:00:09 epoch:    2, train/val loss = 1.38754 / 1.48610, acc =  50.915 /  43.902
00:00:12 epoch:    3, train/val loss = 1.30198 / 1.32772, acc =  57.012 /  53.659
00:00:16 epoch:    4, train/val loss = 1.33467 / 1.21730, acc =  59.451 /  62.195
00:00:19 epoch:    5, train/val loss = 1.13188 / 1.29588, acc =  61.280 /  58.537
00:00:23 epoch:    6, train/val loss = 1.10498 / 1.19576, acc =  62.805 /  54.878
00:00:26 epoch:    7, train/val loss = 0.99980 / 1.19539, acc =  64.024 /  57.317
00:00:29 epoch:    8, train/val loss = 0.99433 / 1.19620, acc =  63.110 /  62.195
00:00:32 epoch:    9, train/val loss = 1.04963 / 1.04080, acc =  66.463 /  65.854
00:00:36 epoch:   10, train/val loss = 0.96301 / 1.11897, acc =  63.720 /  60.976
00:00:39 epoch:   11, train/val loss = 0.96646 / 1.18419, acc =  65.549 /  63.415
00:00:42 epoch: 

In [38]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

Dataset length: 410
Фарафонтова Таисия Михайловна                        :   67
Потанина Александра Викторовна                       :   62
Ивановский Алексей Осипович                          :   48
Крутовский Владимир Михайлович                       :   46
Свентицкая Мария Хрисанфовна                         :   35
Вагнер (Дзвонкевич) Екатерина Николаевна             :   32
Антонова Валентина Константиновна                    :   31
Проскурякова Е. Павловна                             :   30
Семидалов Вениамин Иванович                          :   30
Мендельсон Николай Михайлович                        :   29
train dataset: 328, test dataset: 82

--- test_pipeline:
anchor: torch.Size([64, 3, 300, 300])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-03 18:53:34 siam_window10
load_model: logs/2025-05-03 18:53:34 siam_window10/siam/25 epoch.pt
train/val acc : 80.79267883300781 / 73.17073059082031
Siam model paramet

#### Balanced

In [36]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

train dataset
Фарафонтова Таисия Михайловна   :   53
Потанина Александра Викторовна   :   49
Ивановский Алексей Осипович   :   38
Крутовский Владимир Михайлович   :   36
Свентицкая Мария Хрисанфовна   :   28
Вагнер (Дзвонкевич) Екатерина Николаевна   :   25
Антонова Валентина Константиновна   :   24
Проскурякова Е. Павловна   :   24
Семидалов Вениамин Иванович   :   24
Мендельсон Николай Михайлович   :   23

test dataset
Фарафонтова Таисия Михайловна   :   14
Потанина Александра Викторовна   :   13
Ивановский Алексей Осипович   :   10
Крутовский Владимир Михайлович   :   10
Свентицкая Мария Хрисанфовна   :   7
Вагнер (Дзвонкевич) Екатерина Николаевна   :   7
Антонова Валентина Константиновна   :   7
Проскурякова Е. Павловна   :   6
Семидалов Вениамин Иванович   :   6
Мендельсон Николай Михайлович   :   6

train dataset: 324, test dataset: 86
--- test_pipeline:
anchor: torch.Size([64, 3, 300, 300])
model:   torch.Size([64, 1000])
--- te

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

00:00:08 epoch:    0, train/val loss = 7.84074 / 7.39911, acc =  66.667 /  76.744
00:00:18 epoch:    1, train/val loss = 7.50120 / 7.41082, acc =  75.000 /  69.767
00:00:28 epoch:    2, train/val loss = 7.12177 / 7.24411, acc =  70.988 /  74.419
00:00:38 epoch:    3, train/val loss = 7.45362 / 7.02241, acc =  68.827 /  70.930
00:00:48 epoch:    4, train/val loss = 6.71666 / 6.57740, acc =  70.988 /  73.256
00:00:57 epoch:    5, train/val loss = 6.68457 / 6.01764, acc =  69.136 /  73.256
00:01:07 epoch:    6, train/val loss = 6.82558 / 5.88623, acc =  67.284 /  80.233
00:01:17 epoch:    7, train/val loss = 6.24199 / 5.92330, acc =  69.136 /  68.605
00:01:26 epoch:    8, train/val loss = 6.90740 / 5.07919, acc =  72.531 /  75.581
00:01:36 epoch:    9, train/val loss = 5.83983 / 6.07680, acc =  76.543 /  69.767
00:01:45 epoch:   10, train/val loss = 4.82429 / 6.44050, acc =  78.395 /  75.581
00:01:55 epoch:   11, train/val loss = 5.90996 / 6.60473, acc =  72.222 /  73.256
00:02:05 epoch: 

...

train/val acc : 81.79012298583984 / 76.74418640136719
Siam model parameters number: 0
Class model parameters number: 517642
train dataset: 324, test dataset: 86
--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 10])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

00:00:03 epoch:    0, train/val loss = 2.01985 / 1.70713, acc =  30.247 /  41.860
00:00:06 epoch:    1, train/val loss = 1.66032 / 1.33811, acc =  37.963 /  52.326
00:00:09 epoch:    2, train/val loss = 1.47958 / 1.34305, acc =  53.704 /  61.628
00:00:13 epoch:    3, train/val loss = 1.40496 / 1.32161, acc =  56.481 /  53.488
00:00:15 epoch:    4, train/val loss = 1.24472 / 1.30961, acc =  52.778 /  46.512
00:00:19 epoch:    5, train/val loss = 1.31162 / 1.16719, acc =  53.395 /  62.791
00:00:22 epoch:    6, train/val loss = 1.19016 / 1.05168, acc =  58.025 /  65.116
00:00:25 epoch:    7, train/val loss = 1.08829 / 1.18254, acc =  63.272 /  63.953
00:00:29 epoch:    8, train/val loss = 1.31222 / 1.17206, acc =  62.963 /  62.791
00:00:32 epoch:    9, train/val loss = 1.09020 / 1.25271, acc =  63.889 /  53.488
00:00:35 epoch:   10, train/val loss = 1.26439 / 1.10761, acc =  60.802 /  63.953
00:00:38 epoch:   11, train/val loss = 1.16682 / 1.14141, acc =  63.889 /  61.628
00:00:41 epoch: 

In [39]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

train + test dataset
Фарафонтова Таисия Михайловна                        :   67 = 53 + 14
Потанина Александра Викторовна                       :   62 = 49 + 13
Ивановский Алексей Осипович                          :   48 = 38 + 10
Крутовский Владимир Михайлович                       :   46 = 36 + 10
Свентицкая Мария Хрисанфовна                         :   35 = 28 + 7
Вагнер (Дзвонкевич) Екатерина Николаевна             :   32 = 25 + 7
Антонова Валентина Константиновна                    :   31 = 24 + 7
Проскурякова Е. Павловна                             :   30 = 24 + 6
Семидалов Вениамин Иванович                          :   30 = 24 + 6
Мендельсон Николай Михайлович                        :   29 = 23 + 6
train dataset: 324, test dataset: 86

--- test_pipeline:
anchor: torch.Size([64, 3, 300, 300])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-03 19:00:42 siam_window10_balanced
load_model: logs/2025-05-03 19:00:42

### 20 class

#### Unbalanced 

In [37]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

Dataset length: 639
Фарафонтова Таисия Михайловна   :   67
Потанина Александра Викторовна   :   62
Ивановский Алексей Осипович   :   48
Крутовский Владимир Михайлович   :   46
Свентицкая Мария Хрисанфовна   :   35
Вагнер (Дзвонкевич) Екатерина Николаевна   :   32
Антонова Валентина Константиновна   :   31
Проскурякова Е. Павловна   :   30
Семидалов Вениамин Иванович   :   30
Мендельсон Николай Михайлович   :   29
Крутовская Лидия Симоновна   :   29
Ржавская Надежда Федоровна   :   28
Попов Иван Иванович   :   25
Серошевский Вацлав Леопольдович   :   25
Адрианов Александр Васильевич   :   23
Анучин Дмитрий Николаевич   :   22
Козьмин Николай Николаевич   :   21
Руднев Андрей Дмитриевич   :   19
Пантелеев Лонгин Федорович   :   19
Гуркин Григорий Иванович   :   18

train dataset: 511, test dataset: 128
--- test_pipeline:
anchor: torch.Size([64, 3, 300, 300])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-03 19:07

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

00:00:19 epoch:    0, train/val loss = 7.87160 / 7.68395, acc =  70.254 /  70.312
00:00:35 epoch:    1, train/val loss = 7.59337 / 7.45801, acc =  67.515 /  70.312
00:00:51 epoch:    2, train/val loss = 7.29771 / 7.34110, acc =  68.102 /  68.750
00:01:07 epoch:    3, train/val loss = 7.12495 / 7.00954, acc =  69.080 /  67.188
00:01:23 epoch:    4, train/val loss = 6.54955 / 6.91871, acc =  69.863 /  72.656
00:01:39 epoch:    5, train/val loss = 6.54866 / 6.83909, acc =  71.624 /  68.750
00:01:55 epoch:    6, train/val loss = 5.98392 / 6.16680, acc =  71.820 /  69.531
00:02:10 epoch:    7, train/val loss = 6.28733 / 5.58990, acc =  70.646 /  77.344
00:02:26 epoch:    8, train/val loss = 5.85653 / 6.70120, acc =  74.755 /  68.750
00:02:42 epoch:    9, train/val loss = 5.72281 / 5.87497, acc =  75.342 /  73.438
00:02:58 epoch:   10, train/val loss = 5.64704 / 5.54630, acc =  76.125 /  76.562
00:03:13 epoch:   11, train/val loss = 5.55262 / 5.48743, acc =  76.908 /  75.000
00:03:28 epoch: 

...

train/val acc : 82.97455596923828 / 78.90625
Siam model parameters number: 0
Class model parameters number: 522772
Dataset length: 639
train dataset: 511, test dataset: 128

--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 20])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

00:00:05 epoch:    0, train/val loss = 2.67666 / 2.38804, acc =  20.744 /  27.344
00:00:10 epoch:    1, train/val loss = 2.22753 / 2.14557, acc =  28.767 /  38.281
00:00:15 epoch:    2, train/val loss = 1.93972 / 2.00959, acc =  46.771 /  44.531
00:00:20 epoch:    3, train/val loss = 1.74686 / 1.89403, acc =  51.859 /  48.438
00:00:25 epoch:    4, train/val loss = 1.58238 / 1.85636, acc =  57.143 /  45.312
00:00:30 epoch:    5, train/val loss = 1.48151 / 1.72560, acc =  56.556 /  51.562
00:00:35 epoch:    6, train/val loss = 1.43035 / 1.67158, acc =  57.143 /  51.562
00:00:41 epoch:    7, train/val loss = 1.42395 / 1.57095, acc =  58.904 /  54.688
00:00:46 epoch:    8, train/val loss = 1.31247 / 1.68684, acc =  60.861 /  51.562
00:00:51 epoch:    9, train/val loss = 1.33037 / 1.69345, acc =  61.448 /  50.000
00:00:56 epoch:   10, train/val loss = 1.26041 / 1.62776, acc =  61.840 /  48.438
00:01:01 epoch:   11, train/val loss = 1.23039 / 1.62222, acc =  62.231 /  51.562
00:01:06 epoch: 

In [40]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

Dataset length: 639
Фарафонтова Таисия Михайловна                        :   67
Потанина Александра Викторовна                       :   62
Ивановский Алексей Осипович                          :   48
Крутовский Владимир Михайлович                       :   46
Свентицкая Мария Хрисанфовна                         :   35
Вагнер (Дзвонкевич) Екатерина Николаевна             :   32
Антонова Валентина Константиновна                    :   31
Проскурякова Е. Павловна                             :   30
Семидалов Вениамин Иванович                          :   30
Мендельсон Николай Михайлович                        :   29
Крутовская Лидия Симоновна                           :   29
Ржавская Надежда Федоровна                           :   28
Попов Иван Иванович                                  :   25
Серошевский Вацлав Леопольдович                      :   25
Адрианов Александр Васильевич                        :   23
Анучин Дмитрий Николаевич    

#### Balanced

In [38]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

train dataset
Фарафонтова Таисия Михайловна   :   53
Потанина Александра Викторовна   :   49
Ивановский Алексей Осипович   :   38
Крутовский Владимир Михайлович   :   36
Свентицкая Мария Хрисанфовна   :   28
Вагнер (Дзвонкевич) Екатерина Николаевна   :   25
Антонова Валентина Константиновна   :   24
Проскурякова Е. Павловна   :   24
Семидалов Вениамин Иванович   :   24
Мендельсон Николай Михайлович   :   23
Крутовская Лидия Симоновна   :   23
Ржавская Надежда Федоровна   :   22
Попов Иван Иванович   :   20
Серошевский Вацлав Леопольдович   :   20
Адрианов Александр Васильевич   :   18
Анучин Дмитрий Николаевич   :   17
Козьмин Николай Николаевич   :   16
Руднев Андрей Дмитриевич   :   15
Пантелеев Лонгин Федорович   :   15
Гуркин Григорий Иванович   :   14

test dataset
Фарафонтова Таисия Михайловна   :   14
Потанина Александра Викторовна   :   13
Ивановский Алексей Осипович   :   10
Крутовский Владимир Михайлович   :   10
Свентицкая М

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

00:00:15 epoch:    0, train/val loss = 7.86106 / 7.40164, acc =  70.238 /  75.556
00:00:31 epoch:    1, train/val loss = 7.39997 / 6.55366, acc =  70.833 /  77.037
00:00:46 epoch:    2, train/val loss = 7.27349 / 6.90465, acc =  68.452 /  68.889
00:01:02 epoch:    3, train/val loss = 6.81589 / 6.01608, acc =  71.032 /  74.074
00:01:18 epoch:    4, train/val loss = 6.76081 / 5.58432, acc =  70.040 /  74.074
00:01:34 epoch:    5, train/val loss = 6.09325 / 6.66690, acc =  71.230 /  71.111
00:01:49 epoch:    6, train/val loss = 6.04055 / 6.00596, acc =  72.619 /  69.630
00:02:04 epoch:    7, train/val loss = 6.18826 / 5.83844, acc =  71.230 /  70.370
00:02:20 epoch:    8, train/val loss = 5.82150 / 5.63123, acc =  74.206 /  74.815
00:02:35 epoch:    9, train/val loss = 5.37049 / 6.19307, acc =  77.381 /  73.333
00:02:51 epoch:   10, train/val loss = 5.65950 / 5.37651, acc =  75.397 /  74.815
00:03:07 epoch:   11, train/val loss = 5.44416 / 4.89332, acc =  77.778 /  77.037
00:03:22 epoch: 

...

train/val acc : 75.79365539550781 / 75.55555725097656
Siam model parameters number: 0
Class model parameters number: 522772
train dataset: 504, test dataset: 135
--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 20])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

00:00:05 epoch:    0, train/val loss = 2.80495 / 2.49654, acc =  13.492 /  22.222
00:00:10 epoch:    1, train/val loss = 2.34192 / 2.28354, acc =  30.556 /  34.815
00:00:15 epoch:    2, train/val loss = 2.18993 / 1.84423, acc =  38.095 /  47.407
00:00:20 epoch:    3, train/val loss = 1.92178 / 1.84145, acc =  48.810 /  44.444
00:00:25 epoch:    4, train/val loss = 1.73679 / 1.74989, acc =  51.190 /  51.852
00:00:30 epoch:    5, train/val loss = 1.74600 / 1.67895, acc =  47.222 /  52.593
00:00:35 epoch:    6, train/val loss = 1.62858 / 1.47620, acc =  51.190 /  51.852
00:00:40 epoch:    7, train/val loss = 1.64115 / 1.55490, acc =  50.794 /  53.333
00:00:44 epoch:    8, train/val loss = 1.62694 / 1.38211, acc =  50.000 /  60.000
00:00:49 epoch:    9, train/val loss = 1.49753 / 1.51307, acc =  53.770 /  57.037
00:00:55 epoch:   10, train/val loss = 1.46826 / 1.23792, acc =  54.960 /  54.815
00:01:00 epoch:   11, train/val loss = 1.47578 / 1.37821, acc =  51.587 /  52.593
00:01:05 epoch: 

In [41]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

train + test dataset
Фарафонтова Таисия Михайловна                        :   67 = 53 + 14
Потанина Александра Викторовна                       :   62 = 49 + 13
Ивановский Алексей Осипович                          :   48 = 38 + 10
Крутовский Владимир Михайлович                       :   46 = 36 + 10
Свентицкая Мария Хрисанфовна                         :   35 = 28 + 7
Вагнер (Дзвонкевич) Екатерина Николаевна             :   32 = 25 + 7
Антонова Валентина Константиновна                    :   31 = 24 + 7
Проскурякова Е. Павловна                             :   30 = 24 + 6
Семидалов Вениамин Иванович                          :   30 = 24 + 6
Мендельсон Николай Михайлович                        :   29 = 23 + 6
Крутовская Лидия Симоновна                           :   29 = 23 + 6
Ржавская Надежда Федоровна                           :   28 = 22 + 6
Попов Иван Иванович                                  :   25 = 20 + 5
Серошевский Вацлав Леопольд

### 100 class

#### Unbalanced 

In [39]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

Dataset length: 1384
Фарафонтова Таисия Михайловна   :   67
Потанина Александра Викторовна   :   62
Ивановский Алексей Осипович   :   48
Крутовский Владимир Михайлович   :   46
Свентицкая Мария Хрисанфовна   :   35
Вагнер (Дзвонкевич) Екатерина Николаевна   :   32
Антонова Валентина Константиновна   :   31
Проскурякова Е. Павловна   :   30
Семидалов Вениамин Иванович   :   30
Мендельсон Николай Михайлович   :   29
Крутовская Лидия Симоновна   :   29
Ржавская Надежда Федоровна   :   28
Попов Иван Иванович   :   25
Серошевский Вацлав Леопольдович   :   25
Адрианов Александр Васильевич   :   23
Анучин Дмитрий Николаевич   :   22
Козьмин Николай Николаевич   :   21
Руднев Андрей Дмитриевич   :   19
Пантелеев Лонгин Федорович   :   19
Гуркин Григорий Иванович   :   18
Лаврский Валериан Викторович   :   17
Красноженова Мария Васильевна   :   16
Тиблен Н.   :   16
Сибирякова Анна Михайловна   :   15
Федченко Владимир Иванович   :   15
Леонови

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

00:00:48 epoch:    0, train/val loss = 7.48397 / 7.31481, acc =  71.364 /  68.231
00:01:22 epoch:    1, train/val loss = 6.93885 / 6.38198, acc =  72.087 /  71.841
00:01:57 epoch:    2, train/val loss = 6.31966 / 6.35476, acc =  74.526 /  72.563
00:02:31 epoch:    3, train/val loss = 6.22016 / 6.61719, acc =  72.448 /  72.563
00:03:05 epoch:    4, train/val loss = 5.56242 / 5.87356, acc =  74.797 /  70.397
00:03:40 epoch:    5, train/val loss = 5.53085 / 5.69883, acc =  76.694 /  72.563
00:04:13 epoch:    6, train/val loss = 5.26953 / 5.49472, acc =  76.874 /  75.812
00:04:47 epoch:    7, train/val loss = 5.34480 / 5.71307, acc =  74.977 /  74.007
00:05:21 epoch:    8, train/val loss = 5.48334 / 5.09296, acc =  75.429 /  75.451
00:05:55 epoch:    9, train/val loss = 5.46086 / 5.11035, acc =  73.803 /  75.090
00:06:29 epoch:   10, train/val loss = 5.21859 / 5.06755, acc =  75.790 /  75.451
00:07:04 epoch:   11, train/val loss = 4.86957 / 4.71462, acc =  79.494 /  78.700
00:07:38 epoch: 

...

train/val acc : 79.85546875 / 74.36823272705078
Siam model parameters number: 0
Class model parameters number: 563812
Dataset length: 1384
train dataset: 1107, test dataset: 277

--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 100])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

00:00:12 epoch:    0, train/val loss = 4.21011 / 3.88881, acc =  10.479 /  14.440
00:00:23 epoch:    1, train/val loss = 3.63506 / 3.51781, acc =  18.519 /  22.744
00:00:34 epoch:    2, train/val loss = 3.29551 / 3.36590, acc =  25.113 /  23.827
00:00:46 epoch:    3, train/val loss = 3.09950 / 3.26069, acc =  27.913 /  25.993
00:00:58 epoch:    4, train/val loss = 2.92553 / 3.19121, acc =  30.352 /  27.076
00:01:09 epoch:    5, train/val loss = 2.80477 / 3.17179, acc =  30.352 /  26.715
00:01:21 epoch:    6, train/val loss = 2.69413 / 3.04426, acc =  34.869 /  28.159
00:01:32 epoch:    7, train/val loss = 2.68343 / 2.99939, acc =  34.417 /  31.408
00:01:43 epoch:    8, train/val loss = 2.61658 / 3.02844, acc =  36.495 /  29.242
00:01:55 epoch:    9, train/val loss = 2.54665 / 2.92703, acc =  35.140 /  32.852
00:02:06 epoch:   10, train/val loss = 2.51349 / 2.92444, acc =  35.772 /  33.213
00:02:17 epoch:   11, train/val loss = 2.47357 / 2.91854, acc =  37.037 /  35.018
00:02:29 epoch: 

In [36]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

Dataset length: 1384
Фарафонтова Таисия Михайловна                        :   67
Потанина Александра Викторовна                       :   62
Ивановский Алексей Осипович                          :   48
Крутовский Владимир Михайлович                       :   46
Свентицкая Мария Хрисанфовна                         :   35
Вагнер (Дзвонкевич) Екатерина Николаевна             :   32
Антонова Валентина Константиновна                    :   31
Проскурякова Е. Павловна                             :   30
Семидалов Вениамин Иванович                          :   30
Мендельсон Николай Михайлович                        :   29
Крутовская Лидия Симоновна                           :   29
Ржавская Надежда Федоровна                           :   28
Попов Иван Иванович                                  :   25
Серошевский Вацлав Леопольдович                      :   25
Адрианов Александр Васильевич                        :   23
Анучин Дмитрий Николаевич   

#### Balanced

In [40]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

train dataset
Фарафонтова Таисия Михайловна   :   53
Потанина Александра Викторовна   :   49
Ивановский Алексей Осипович   :   38
Крутовский Владимир Михайлович   :   36
Свентицкая Мария Хрисанфовна   :   28
Вагнер (Дзвонкевич) Екатерина Николаевна   :   25
Антонова Валентина Константиновна   :   24
Проскурякова Е. Павловна   :   24
Семидалов Вениамин Иванович   :   24
Мендельсон Николай Михайлович   :   23
Крутовская Лидия Симоновна   :   23
Ржавская Надежда Федоровна   :   22
Попов Иван Иванович   :   20
Серошевский Вацлав Леопольдович   :   20
Адрианов Александр Васильевич   :   18
Анучин Дмитрий Николаевич   :   17
Козьмин Николай Николаевич   :   16
Руднев Андрей Дмитриевич   :   15
Пантелеев Лонгин Федорович   :   15
Гуркин Григорий Иванович   :   14
Лаврский Валериан Викторович   :   13
Красноженова Мария Васильевна   :   12
Тиблен Н.   :   12
Сибирякова Анна Михайловна   :   12
Федченко Владимир Иванович   :   12
Леонович (Анга

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

00:00:32 epoch:    0, train/val loss = 7.49577 / 6.93327, acc =  70.113 /  76.562
00:01:04 epoch:    1, train/val loss = 6.76056 / 6.23155, acc =  73.590 /  73.750
00:01:38 epoch:    2, train/val loss = 5.91343 / 5.59123, acc =  75.376 /  74.062
00:02:11 epoch:    3, train/val loss = 5.86615 / 4.77383, acc =  74.436 /  79.375
00:02:45 epoch:    4, train/val loss = 5.96425 / 5.23270, acc =  73.872 /  75.000
00:03:19 epoch:    5, train/val loss = 5.41244 / 4.45514, acc =  76.410 /  79.688
00:03:53 epoch:    6, train/val loss = 5.68289 / 4.05667, acc =  74.530 /  84.062
00:04:27 epoch:    7, train/val loss = 5.25921 / 4.48372, acc =  76.128 /  80.625
00:05:01 epoch:    8, train/val loss = 4.94937 / 4.81348, acc =  77.444 /  76.562
00:05:35 epoch:    9, train/val loss = 5.03780 / 4.99886, acc =  77.068 /  77.500
00:06:08 epoch:   10, train/val loss = 5.24983 / 4.70153, acc =  75.094 /  77.500
00:06:41 epoch:   11, train/val loss = 5.13269 / 4.33219, acc =  74.812 /  80.312
00:07:15 epoch: 

...

train/val acc : 76.78571319580078 / 82.5
Siam model parameters number: 0
Class model parameters number: 563812
train dataset: 1064, test dataset: 320
--- test_pipeline:
X_batch: torch.Size([64, 3, 300, 300])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 100])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

00:00:12 epoch:    0, train/val loss = 4.43344 / 4.39644, acc =   4.323 /   4.688
00:00:24 epoch:    1, train/val loss = 4.05779 / 3.93885, acc =   8.929 /  12.188
00:00:36 epoch:    2, train/val loss = 3.65468 / 3.87775, acc =  15.789 /  12.188
00:00:48 epoch:    3, train/val loss = 3.41274 / 3.56458, acc =  16.259 /  16.875
00:01:00 epoch:    4, train/val loss = 3.18649 / 3.71738, acc =  22.650 /  15.937
00:01:12 epoch:    5, train/val loss = 3.03545 / 3.62544, acc =  25.658 /  17.188
00:01:24 epoch:    6, train/val loss = 3.01391 / 3.67306, acc =  23.778 /  21.875
00:01:36 epoch:    7, train/val loss = 2.86736 / 3.48206, acc =  26.786 /  19.688
00:01:47 epoch:    8, train/val loss = 2.86564 / 3.58260, acc =  25.846 /  20.000
00:02:00 epoch:    9, train/val loss = 2.76178 / 3.48750, acc =  31.297 /  25.625
00:02:11 epoch:   10, train/val loss = 2.80573 / 3.34369, acc =  28.665 /  24.688
00:02:23 epoch:   11, train/val loss = 2.67481 / 3.52184, acc =  31.861 /  23.125
00:02:35 epoch: 

In [37]:
config = {
    "device" : DEVICE,

    "type" : "window",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_bin",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment(config)

Device: cuda
Siam model parameters number: 513000

train + test dataset
Фарафонтова Таисия Михайловна                        :   67 = 53 + 14
Потанина Александра Викторовна                       :   62 = 49 + 13
Ивановский Алексей Осипович                          :   48 = 38 + 10
Крутовский Владимир Михайлович                       :   46 = 36 + 10
Свентицкая Мария Хрисанфовна                         :   35 = 28 + 7
Вагнер (Дзвонкевич) Екатерина Николаевна             :   32 = 25 + 7
Антонова Валентина Константиновна                    :   31 = 24 + 7
Проскурякова Е. Павловна                             :   30 = 24 + 6
Семидалов Вениамин Иванович                          :   30 = 24 + 6
Мендельсон Николай Михайлович                        :   29 = 23 + 6
Крутовская Лидия Симоновна                           :   29 = 23 + 6
Ржавская Надежда Федоровна                           :   28 = 22 + 6
Попов Иван Иванович                                  :   25 = 20 + 5
Серошевский Вацлав Леопольд

## Strokes

### 3 class

#### Unbalanced 

In [43]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

Dataset length: 2865
Потанина Александра Викторовна   :   1384
Фарафонтова Таисия Михайловна   :   775
Вагнер (Дзвонкевич) Екатерина Николаевна   :   706

train dataset: 2292, test dataset: 573
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 977])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-03 20:17:18 siam_strokes3
Make info/2025-05-03 20:17:18 siam_strokes3

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:25 epoch:    0, train/val loss = 8.97960 / 8.96625, acc =  54.625 /  55.323
00:00:36 epoch:    1, train/val loss = 8.52975 / 8.39989, acc =  61.736 /  54.974
00:00:46 epoch:    2, train/val loss = 8.68000 / 7.97536, acc =  56.414 /  63.176
00:00:56 epoch:    3, train/val loss = 7.03414 / 5.86673, acc =  66.056 /  70.506
00:01:05 epoch:    4, train/val loss = 5.62804 / 5.01738, acc =  72.993 /  75.742
00:01:15 epoch:    5, train/val loss = 5.40361 / 4.83141, acc =  73.298 /  77.138
00:01:25 epoch:    6, train/val loss = 5.13555 / 4.92055, acc =  75.873 /  78.010
00:01:35 epoch:    7, train/val loss = 5.21533 / 4.91024, acc =  75.349 /  78.010
00:01:44 epoch:    8, train/val loss = 4.89569 / 4.57891, acc =  76.440 /  79.581
00:01:53 epoch:    9, train/val loss = 4.97866 / 4.63551, acc =  75.567 /  77.661
00:02:03 epoch:   10, train/val loss = 4.81028 / 4.68119, acc =  77.661 /  80.279
00:02:12 epoch:   11, train/val loss = 4.61268 / 4.31923, acc =  77.749 /  79.407
00:02:21 epoch: 

...

train/val acc : 84.7294921875 / 85.86387634277344
Siam model parameters number: 0
Class model parameters number: 514051
Dataset length: 2865
train dataset: 2292, test dataset: 573

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 1021])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:03 epoch:    0, train/val loss = 0.54802 / 0.47528, acc =  81.981 /  84.642
00:00:06 epoch:    1, train/val loss = 0.40274 / 0.45822, acc =  85.864 /  83.246
00:00:09 epoch:    2, train/val loss = 0.38595 / 0.45414, acc =  86.126 /  84.293
00:00:13 epoch:    3, train/val loss = 0.37831 / 0.44750, acc =  86.736 /  83.770
00:00:16 epoch:    4, train/val loss = 0.37380 / 0.43590, acc =  86.867 /  84.468
00:00:19 epoch:    5, train/val loss = 0.37351 / 0.45532, acc =  86.824 /  84.817
00:00:23 epoch:    6, train/val loss = 0.37134 / 0.44514, acc =  86.911 /  84.991
00:00:27 epoch:    7, train/val loss = 0.37685 / 0.43377, acc =  86.911 /  84.468
00:00:31 epoch:    8, train/val loss = 0.36753 / 0.43294, acc =  86.606 /  84.293
00:00:35 epoch:    9, train/val loss = 0.37308 / 0.42775, acc =  86.693 /  84.642
00:00:40 epoch:   10, train/val loss = 0.37053 / 0.45283, acc =  86.867 /  83.595
00:00:44 epoch:   11, train/val loss = 0.36993 / 0.43632, acc =  87.173 /  84.119
00:00:48 epoch: 

In [44]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

Dataset length: 2865
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
train dataset: 2292, test dataset: 573

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 741])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-03 20:17:18 siam_strokes3
load_model: logs/2025-05-03 20:17:18 siam_strokes3/siam/29 epoch.pt
train/val acc : 84.20594024658203 / 86.2129135131836
Siam model parameters number: 0
Class model parameters number: 514051
Dataset length: 2865
train dataset: 2292, test dataset: 573

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 977])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

load_model: logs/2025-05-03 20:17:18 siam_strokes3/class/6 epoch.pt
train/val acc : 86.78010702133179 / 87.95811533927917
hits@k: {1: 0.86

#### Balanced

In [44]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

train dataset
Потанина Александра Викторовна   :   1107
Фарафонтова Таисия Михайловна   :   620
Вагнер (Дзвонкевич) Екатерина Николаевна   :   564

test dataset
Потанина Александра Викторовна   :   277
Фарафонтова Таисия Михайловна   :   155
Вагнер (Дзвонкевич) Екатерина Николаевна   :   142

train dataset: 2291, test dataset: 574
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 750])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-03 20:24:35 siam_strokes3_balanced
Make info/2025-05-03 20:24:35 siam_strokes3_balanced

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:09 epoch:    0, train/val loss = 7.94419 / 5.63896, acc =  66.521 /  73.868
00:00:19 epoch:    1, train/val loss = 5.86720 / 5.11415, acc =  71.846 /  75.261
00:00:28 epoch:    2, train/val loss = 5.61726 / 4.98555, acc =  73.243 /  77.352
00:00:37 epoch:    3, train/val loss = 5.53552 / 5.09262, acc =  73.025 /  76.481
00:00:47 epoch:    4, train/val loss = 5.29914 / 4.62867, acc =  74.596 /  76.132
00:00:56 epoch:    5, train/val loss = 5.27116 / 4.51036, acc =  75.207 /  79.443
00:01:06 epoch:    6, train/val loss = 5.09773 / 4.80706, acc =  76.560 /  78.571
00:01:16 epoch:    7, train/val loss = 4.72047 / 4.43680, acc =  78.088 /  79.965
00:01:25 epoch:    8, train/val loss = 4.64359 / 3.78088, acc =  77.608 /  82.753
00:01:35 epoch:    9, train/val loss = 4.42362 / 3.62509, acc =  79.398 /  81.882
00:01:44 epoch:   10, train/val loss = 4.29903 / 3.62123, acc =  80.576 /  84.146
00:01:53 epoch:   11, train/val loss = 4.30684 / 3.84216, acc =  80.663 /  82.578
00:02:03 epoch: 

...

train/val acc : 87.9528579711914 / 88.67595672607422
Siam model parameters number: 0
Class model parameters number: 514051
train dataset: 2291, test dataset: 574
--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 800])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:03 epoch:    0, train/val loss = 0.43594 / 0.37408, acc =  84.112 /  85.366
00:00:06 epoch:    1, train/val loss = 0.36495 / 0.35720, acc =  87.735 /  87.108
00:00:10 epoch:    2, train/val loss = 0.34275 / 0.30332, acc =  88.608 /  88.676
00:00:13 epoch:    3, train/val loss = 0.37717 / 0.28438, acc =  86.774 /  91.115
00:00:16 epoch:    4, train/val loss = 0.32775 / 0.33076, acc =  89.350 /  88.328
00:00:20 epoch:    5, train/val loss = 0.33157 / 0.35129, acc =  89.000 /  86.934
00:00:23 epoch:    6, train/val loss = 0.31664 / 0.32935, acc =  88.695 /  90.418
00:00:27 epoch:    7, train/val loss = 0.32955 / 0.31445, acc =  88.782 /  89.199
00:00:30 epoch:    8, train/val loss = 0.34964 / 0.30612, acc =  87.909 /  91.115
00:00:34 epoch:    9, train/val loss = 0.30199 / 0.35344, acc =  89.699 /  86.760
00:00:37 epoch:   10, train/val loss = 0.34229 / 0.32079, acc =  87.778 /  91.115
00:00:41 epoch:   11, train/val loss = 0.33019 / 0.31575, acc =  88.739 /  90.244
00:00:44 epoch: 

In [45]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
train dataset: 2291, test dataset: 574

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 726])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-03 20:24:35 siam_strokes3_balanced
load_model: logs/2025-05-03 20:24:35 siam_strokes3_balanced/siam/26 epoch.pt
train/val acc : 86.86162567138672 / 87.4564437866211
Siam model parameters number: 0
Class model parameters number: 514051
train dataset: 2291, test dataset: 574

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 1121])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

load_model: logs/2025-05-03 20:24:35 siam_strokes3_balanced/class/20 epoch.pt
train/val acc : 89.742469

### 10 class

#### Unbalanced 

In [45]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

Dataset length: 6670
Потанина Александра Викторовна   :   1384
Фарафонтова Таисия Михайловна   :   775
Вагнер (Дзвонкевич) Екатерина Николаевна   :   706
Крутовский Владимир Михайлович   :   678
Ивановский Алексей Осипович   :   664
Антонова Валентина Константиновна   :   544
Семидалов Вениамин Иванович   :   518
Проскурякова Е. Павловна   :   489
Крутовская Лидия Симоновна   :   462
Ржавская Надежда Федоровна   :   450

train dataset: 5336, test dataset: 1334
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 748])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-03 20:31:27 siam_strokes10
Make info/2025-05-03 20:31:27 siam_strokes10

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:01:14 epoch:    0, train/val loss = 8.49930 / 7.58653, acc =  58.302 /  60.720
00:01:48 epoch:    1, train/val loss = 7.53563 / 7.51825, acc =  60.982 /  62.294
00:02:23 epoch:    2, train/val loss = 7.57100 / 7.26915, acc =  61.057 /  62.819
00:02:58 epoch:    3, train/val loss = 7.43372 / 7.14954, acc =  61.788 /  63.868
00:03:31 epoch:    4, train/val loss = 7.19602 / 6.97068, acc =  63.325 /  64.168
00:04:04 epoch:    5, train/val loss = 7.10520 / 6.90347, acc =  63.831 /  65.667
00:04:36 epoch:    6, train/val loss = 6.89218 / 6.66179, acc =  65.667 /  65.517
00:05:10 epoch:    7, train/val loss = 6.69439 / 6.31383, acc =  66.136 /  68.516
00:05:42 epoch:    8, train/val loss = 6.41115 / 6.49999, acc =  68.441 /  66.942
00:06:15 epoch:    9, train/val loss = 6.27006 / 5.96228, acc =  68.347 /  71.139
00:06:48 epoch:   10, train/val loss = 6.32532 / 6.42401, acc =  68.759 /  68.741
00:07:21 epoch:   11, train/val loss = 6.13902 / 6.32178, acc =  70.558 /  69.415
00:07:55 epoch: 

...

train/val acc : 74.40029907226562 / 75.03748321533203
Siam model parameters number: 0
Class model parameters number: 517642
Dataset length: 6670
train dataset: 5336, test dataset: 1334

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 1050])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 10])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:10 epoch:    0, train/val loss = 1.60875 / 1.51710, acc =  42.879 /  46.402
00:00:21 epoch:    1, train/val loss = 1.51096 / 1.49392, acc =  46.214 /  47.376
00:00:32 epoch:    2, train/val loss = 1.48809 / 1.45010, acc =  47.076 /  48.651
00:00:42 epoch:    3, train/val loss = 1.47386 / 1.44737, acc =  47.320 /  48.126
00:00:53 epoch:    4, train/val loss = 1.45426 / 1.44368, acc =  47.545 /  47.451
00:01:05 epoch:    5, train/val loss = 1.45370 / 1.41419, acc =  47.620 /  49.325
00:01:16 epoch:    6, train/val loss = 1.44123 / 1.42880, acc =  48.145 /  49.400
00:01:26 epoch:    7, train/val loss = 1.44019 / 1.42373, acc =  48.557 /  49.775
00:01:37 epoch:    8, train/val loss = 1.43577 / 1.41340, acc =  47.939 /  50.375
00:01:48 epoch:    9, train/val loss = 1.42873 / 1.42911, acc =  48.632 /  49.550
00:01:58 epoch:   10, train/val loss = 1.42683 / 1.42852, acc =  48.276 /  51.199
00:02:09 epoch:   11, train/val loss = 1.42151 / 1.39923, acc =  49.325 /  50.750
00:02:20 epoch: 

In [46]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

Dataset length: 6670
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
train dataset: 5336, test dataset: 1334

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 928])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-03 20:31:27 siam_strokes10
load_model: logs/2025-05-03 20:31:27 siam_strokes10/siam/27 epoch.pt
train/val acc : 73.95053100585938 / 73.98800659179688
Si

#### Balanced

In [46]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

train dataset
Потанина Александра Викторовна   :   1107
Фарафонтова Таисия Михайловна   :   620
Вагнер (Дзвонкевич) Екатерина Николаевна   :   564
Крутовский Владимир Михайлович   :   542
Ивановский Алексей Осипович   :   531
Антонова Валентина Константиновна   :   435
Семидалов Вениамин Иванович   :   414
Проскурякова Е. Павловна   :   391
Крутовская Лидия Симоновна   :   369
Ржавская Надежда Федоровна   :   360

test dataset
Потанина Александра Викторовна   :   277
Фарафонтова Таисия Михайловна   :   155
Вагнер (Дзвонкевич) Екатерина Николаевна   :   142
Крутовский Владимир Михайлович   :   136
Ивановский Алексей Осипович   :   133
Антонова Валентина Константиновна   :   109
Семидалов Вениамин Иванович   :   104
Проскурякова Е. Павловна   :   98
Крутовская Лидия Симоновна   :   93
Ржавская Надежда Федоровна   :   90

train dataset: 5333, test dataset: 1337
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 1178])
model:   torch.Size([

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:32 epoch:    0, train/val loss = 8.78203 / 8.20675, acc =  55.035 /  55.797
00:01:06 epoch:    1, train/val loss = 7.62680 / 7.34069, acc =  61.616 /  62.528
00:01:39 epoch:    2, train/val loss = 7.38371 / 7.31264, acc =  62.010 /  63.201
00:02:14 epoch:    3, train/val loss = 7.35621 / 7.27899, acc =  61.748 /  62.603
00:02:47 epoch:    4, train/val loss = 7.20717 / 7.24795, acc =  62.198 /  62.079
00:03:21 epoch:    5, train/val loss = 7.11131 / 7.41185, acc =  63.941 /  60.808
00:03:53 epoch:    6, train/val loss = 6.93766 / 6.81001, acc =  64.560 /  66.417
00:04:26 epoch:    7, train/val loss = 7.12096 / 7.26068, acc =  63.566 /  62.977
00:04:58 epoch:    8, train/val loss = 6.91500 / 6.68765, acc =  65.423 /  65.894
00:05:30 epoch:    9, train/val loss = 6.90283 / 7.02206, acc =  64.785 /  65.071
00:06:03 epoch:   10, train/val loss = 6.94204 / 7.21503, acc =  64.542 /  63.949
00:06:36 epoch:   11, train/val loss = 7.10232 / 7.65707, acc =  64.729 /  61.481
00:07:09 epoch: 

...

train/val acc : 75.4547119140625 / 76.51458740234375
Siam model parameters number: 0
Class model parameters number: 517642
train dataset: 5333, test dataset: 1337
--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 789])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 10])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:10 epoch:    0, train/val loss = 1.58332 / 1.50227, acc =  41.928 /  45.924
00:00:20 epoch:    1, train/val loss = 1.46155 / 1.49930, acc =  46.822 /  47.345
00:00:31 epoch:    2, train/val loss = 1.44813 / 1.45020, acc =  47.497 /  46.821
00:00:41 epoch:    3, train/val loss = 1.41244 / 1.36141, acc =  49.203 /  49.663
00:00:52 epoch:    4, train/val loss = 1.38548 / 1.41622, acc =  50.741 /  48.093
00:01:02 epoch:    5, train/val loss = 1.37029 / 1.35590, acc =  51.003 /  48.392
00:01:12 epoch:    6, train/val loss = 1.38121 / 1.39946, acc =  50.572 /  50.935
00:01:23 epoch:    7, train/val loss = 1.37725 / 1.34264, acc =  51.453 /  52.954
00:01:34 epoch:    8, train/val loss = 1.37264 / 1.42987, acc =  50.909 /  49.663
00:01:44 epoch:    9, train/val loss = 1.37703 / 1.35593, acc =  51.078 /  48.616
00:01:55 epoch:   10, train/val loss = 1.35885 / 1.33433, acc =  51.378 /  51.533
00:02:06 epoch:   11, train/val loss = 1.36808 / 1.34903, acc =  51.097 /  52.805
00:02:16 epoch: 

In [47]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
train dataset: 5333, test dataset: 1337

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 744])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-03 20:54:59 siam_strokes10_balance

### 20 class

#### Unbalanced 

In [47]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

Dataset length: 10216
Потанина Александра Викторовна   :   1384
Фарафонтова Таисия Михайловна   :   775
Вагнер (Дзвонкевич) Екатерина Николаевна   :   706
Крутовский Владимир Михайлович   :   678
Ивановский Алексей Осипович   :   664
Антонова Валентина Константиновна   :   544
Семидалов Вениамин Иванович   :   518
Проскурякова Е. Павловна   :   489
Крутовская Лидия Симоновна   :   462
Ржавская Надежда Федоровна   :   450
Мендельсон Николай Михайлович   :   426
Серошевский Вацлав Леопольдович   :   412
Попов Иван Иванович   :   404
Свентицкая Мария Хрисанфовна   :   392
Адрианов Александр Васильевич   :   382
Анучин Дмитрий Николаевич   :   369
Гуркин Григорий Иванович   :   306
Лаврский Валериан Викторович   :   288
Козьмин Николай Николаевич   :   285
Лаврский Константин Викторович   :   282

train dataset: 8172, test dataset: 2044
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 779])
model:   torch.Size([64, 1000])
--- test_pipelin

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

00:01:43 epoch:    0, train/val loss = 8.15318 / 7.44117, acc =  59.178 /  62.916
00:02:43 epoch:    1, train/val loss = 7.43034 / 7.26224, acc =  62.237 /  62.867
00:03:42 epoch:    2, train/val loss = 7.16998 / 7.26561, acc =  63.155 /  64.481
00:04:42 epoch:    3, train/val loss = 6.87110 / 6.88686, acc =  65.663 /  64.775
00:05:40 epoch:    4, train/val loss = 6.67264 / 6.84361, acc =  66.593 /  66.243
00:06:38 epoch:    5, train/val loss = 6.55270 / 6.30584, acc =  67.168 /  68.591
00:07:36 epoch:    6, train/val loss = 6.51319 / 6.48493, acc =  67.352 /  68.004
00:08:34 epoch:    7, train/val loss = 6.41426 / 6.31227, acc =  67.597 /  68.738
00:09:31 epoch:    8, train/val loss = 6.25152 / 6.35205, acc =  69.041 /  68.297
00:10:30 epoch:    9, train/val loss = 6.18576 / 6.05907, acc =  69.383 /  70.450
00:11:27 epoch:   10, train/val loss = 6.01573 / 5.99202, acc =  70.754 /  70.254
00:12:25 epoch:   11, train/val loss = 6.09122 / 6.02216, acc =  70.056 /  70.744
00:13:24 epoch: 

...

train/val acc : 74.75526428222656 / 74.85322570800781
Siam model parameters number: 0
Class model parameters number: 522772
Dataset length: 10216
train dataset: 8172, test dataset: 2044

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 1011])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 20])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

00:00:20 epoch:    0, train/val loss = 2.17697 / 2.02659, acc =  28.744 /  33.708
00:00:38 epoch:    1, train/val loss = 2.05472 / 1.98379, acc =  32.073 /  32.975
00:00:58 epoch:    2, train/val loss = 2.02228 / 1.97459, acc =  33.040 /  33.464
00:01:17 epoch:    3, train/val loss = 2.00601 / 1.94663, acc =  33.762 /  36.301
00:01:36 epoch:    4, train/val loss = 1.98485 / 1.95350, acc =  34.104 /  35.372
00:01:55 epoch:    5, train/val loss = 1.96492 / 1.92104, acc =  34.924 /  37.965
00:02:14 epoch:    6, train/val loss = 1.95643 / 1.90847, acc =  35.463 /  37.084
00:02:34 epoch:    7, train/val loss = 1.94530 / 1.90633, acc =  35.597 /  36.546
00:02:53 epoch:    8, train/val loss = 1.93699 / 1.90776, acc =  35.720 /  36.497
00:03:12 epoch:    9, train/val loss = 1.92689 / 1.89333, acc =  35.414 /  37.671
00:03:31 epoch:   10, train/val loss = 1.91105 / 1.85872, acc =  36.980 /  38.307
00:03:50 epoch:   11, train/val loss = 1.89780 / 1.86368, acc =  36.686 /  39.090
00:04:08 epoch: 

In [48]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

Dataset length: 10216
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
Мендельсон Николай Михайлович                        :   426
Серошевский Вацлав Леопольдович                      :   412
Попов Иван Иванович                                  :   404
Свентицкая Мария Хрисанфовна                         :   392
Адрианов Александр Васильевич                        :   382
Анучин Дмит

#### Balanced

In [48]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

train dataset
Потанина Александра Викторовна   :   1107
Фарафонтова Таисия Михайловна   :   620
Вагнер (Дзвонкевич) Екатерина Николаевна   :   564
Крутовский Владимир Михайлович   :   542
Ивановский Алексей Осипович   :   531
Антонова Валентина Константиновна   :   435
Семидалов Вениамин Иванович   :   414
Проскурякова Е. Павловна   :   391
Крутовская Лидия Симоновна   :   369
Ржавская Надежда Федоровна   :   360
Мендельсон Николай Михайлович   :   340
Серошевский Вацлав Леопольдович   :   329
Попов Иван Иванович   :   323
Свентицкая Мария Хрисанфовна   :   313
Адрианов Александр Васильевич   :   305
Анучин Дмитрий Николаевич   :   295
Гуркин Григорий Иванович   :   244
Лаврский Валериан Викторович   :   230
Козьмин Николай Николаевич   :   228
Лаврский Константин Викторович   :   225

test dataset
Потанина Александра Викторовна   :   277
Фарафонтова Таисия Михайловна   :   155
Вагнер (Дзвонкевич) Екатерина Николаевна   :   142
Крутовс

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

00:00:58 epoch:    0, train/val loss = 8.09047 / 7.25021, acc =  59.608 /  62.701
00:01:55 epoch:    1, train/val loss = 7.30572 / 7.18961, acc =  62.290 /  65.578
00:02:52 epoch:    2, train/val loss = 6.93626 / 6.73607, acc =  65.279 /  67.674
00:03:49 epoch:    3, train/val loss = 6.31904 / 6.61777, acc =  68.500 /  66.358
00:04:46 epoch:    4, train/val loss = 6.30909 / 6.46404, acc =  68.855 /  68.308
00:05:42 epoch:    5, train/val loss = 6.36586 / 6.10514, acc =  68.634 /  70.551
00:06:39 epoch:    6, train/val loss = 6.01441 / 5.94635, acc =  70.961 /  70.600
00:07:38 epoch:    7, train/val loss = 5.88447 / 6.23329, acc =  71.255 /  69.576
00:08:36 epoch:    8, train/val loss = 5.89766 / 6.31778, acc =  71.329 /  69.381
00:09:34 epoch:    9, train/val loss = 5.79915 / 5.80370, acc =  71.966 /  70.746
00:10:31 epoch:   10, train/val loss = 5.76130 / 6.38664, acc =  72.456 /  68.454
00:11:27 epoch:   11, train/val loss = 5.92511 / 6.04274, acc =  71.072 /  70.405
00:12:25 epoch: 

...

train/val acc : 74.81934356689453 / 72.5987319946289
Siam model parameters number: 0
Class model parameters number: 522772
train dataset: 8165, test dataset: 2051
--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 1084])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 20])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

00:00:19 epoch:    0, train/val loss = 2.40074 / 2.25297, acc =  20.563 /  24.135
00:00:38 epoch:    1, train/val loss = 2.23680 / 2.15923, acc =  26.283 /  25.646
00:00:57 epoch:    2, train/val loss = 2.16123 / 2.04963, acc =  28.047 /  33.106
00:01:16 epoch:    3, train/val loss = 2.11067 / 2.06020, acc =  30.288 /  28.523
00:01:35 epoch:    4, train/val loss = 2.09422 / 2.01324, acc =  29.982 /  29.985
00:01:54 epoch:    5, train/val loss = 2.07566 / 2.02437, acc =  29.810 /  32.423
00:02:13 epoch:    6, train/val loss = 2.04640 / 2.08518, acc =  30.863 /  32.521
00:02:32 epoch:    7, train/val loss = 2.04692 / 1.97554, acc =  31.439 /  33.350
00:02:51 epoch:    8, train/val loss = 2.06332 / 2.06587, acc =  31.108 /  28.766
00:03:10 epoch:    9, train/val loss = 2.01689 / 1.96104, acc =  32.590 /  33.155
00:03:29 epoch:   10, train/val loss = 1.99504 / 1.96600, acc =  33.631 /  33.740
00:03:49 epoch:   11, train/val loss = 1.99466 / 1.95057, acc =  33.337 /  34.666
00:04:08 epoch: 

In [49]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
Мендельсон Николай Михайлович                        :   426 = 340 + 86
Серошевский Вацлав Леопольдович                      :   412 = 329 + 83
Попов Иван Иванович                                  : 

### 100 class

#### Unbalanced 

In [ ]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

Dataset length: 20909
Потанина Александра Викторовна   :   1384
Фарафонтова Таисия Михайловна   :   775
Вагнер (Дзвонкевич) Екатерина Николаевна   :   706
Крутовский Владимир Михайлович   :   678
Ивановский Алексей Осипович   :   664
Антонова Валентина Константиновна   :   544
Семидалов Вениамин Иванович   :   518
Проскурякова Е. Павловна   :   489
Крутовская Лидия Симоновна   :   462
Ржавская Надежда Федоровна   :   450
Мендельсон Николай Михайлович   :   426
Серошевский Вацлав Леопольдович   :   412
Попов Иван Иванович   :   404
Свентицкая Мария Хрисанфовна   :   392
Адрианов Александр Васильевич   :   382
Анучин Дмитрий Николаевич   :   369
Гуркин Григорий Иванович   :   306
Лаврский Валериан Викторович   :   288
Козьмин Николай Николаевич   :   285
Лаврский Константин Викторович   :   282
Костюрина Мария Николаевна   :   258
Руднев Андрей Дмитриевич   :   258
Педашенко-Третьякова Мария Ивановна   :   239
Красноженова Мария Васильев

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:01:40 epoch:    0, train/val loss = 7.86288 / 7.48854, acc =  60.244 /  62.147
00:03:15 epoch:    1, train/val loss = 7.27153 / 7.20946, acc =  63.203 /  63.702
00:04:50 epoch:    2, train/val loss = 7.13890 / 7.14797, acc =  63.974 /  63.486
00:06:23 epoch:    3, train/val loss = 7.01649 / 7.03758, acc =  64.955 /  65.423
00:07:56 epoch:    4, train/val loss = 6.90596 / 7.01435, acc =  65.260 /  64.682
00:09:30 epoch:    5, train/val loss = 6.85352 / 6.88906, acc =  65.738 /  66.595
00:11:04 epoch:    6, train/val loss = 6.69534 / 6.83128, acc =  66.760 /  66.141
00:12:39 epoch:    7, train/val loss = 6.57397 / 6.61416, acc =  67.137 /  67.408
00:14:16 epoch:    8, train/val loss = 6.56944 / 6.51476, acc =  67.681 /  67.623
00:15:57 epoch:    9, train/val loss = 6.43564 / 6.38399, acc =  67.878 /  68.077
00:17:33 epoch:   10, train/val loss = 6.25254 / 6.30177, acc =  69.540 /  69.273
00:19:08 epoch:   11, train/val loss = 6.26772 / 6.27436, acc =  68.883 /  68.962
00:20:46 epoch: 

...

train/val acc : 75.70993041992188 / 74.86848449707031
Siam model parameters number: 0
Class model parameters number: 563812
Dataset length: 20909
train dataset: 16727, test dataset: 4182

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 742])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 100])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:00:34 epoch:    0, train/val loss = 3.32224 / 3.06805, acc =  21.397 /  24.055
00:01:07 epoch:    1, train/val loss = 3.03723 / 2.99820, acc =  24.565 /  25.538
00:01:40 epoch:    2, train/val loss = 2.95811 / 2.95098, acc =  25.755 /  26.351
00:02:13 epoch:    3, train/val loss = 2.90878 / 2.94378, acc =  26.102 /  25.992
00:02:46 epoch:    4, train/val loss = 2.87159 / 2.91009, acc =  26.938 /  27.953
00:03:18 epoch:    5, train/val loss = 2.83886 / 2.89589, acc =  27.435 /  27.929
00:03:50 epoch:    6, train/val loss = 2.80854 / 2.89510, acc =  27.548 /  27.116
00:04:22 epoch:    7, train/val loss = 2.78368 / 2.87867, acc =  27.931 /  27.881
00:04:54 epoch:    8, train/val loss = 2.75624 / 2.86345, acc =  28.367 /  28.121
00:05:27 epoch:    9, train/val loss = 2.73823 / 2.86266, acc =  29.115 /  28.360
00:05:59 epoch:   10, train/val loss = 2.71565 / 2.84158, acc =  29.144 /  28.957
00:06:31 epoch:   11, train/val loss = 2.69327 / 2.84030, acc =  29.808 /  28.981
00:07:03 epoch: 

In [38]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

Dataset length: 20909
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
Мендельсон Николай Михайлович                        :   426
Серошевский Вацлав Леопольдович                      :   412
Попов Иван Иванович                                  :   404
Свентицкая Мария Хрисанфовна                         :   392
Адрианов Александр Васильевич                        :   382
Анучин Дмит

#### Balanced

In [20]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

train dataset
Потанина Александра Викторовна   :   1107
Фарафонтова Таисия Михайловна   :   620
Вагнер (Дзвонкевич) Екатерина Николаевна   :   564
Крутовский Владимир Михайлович   :   542
Ивановский Алексей Осипович   :   531
Антонова Валентина Константиновна   :   435
Семидалов Вениамин Иванович   :   414
Проскурякова Е. Павловна   :   391
Крутовская Лидия Симоновна   :   369
Ржавская Надежда Федоровна   :   360
Мендельсон Николай Михайлович   :   340
Серошевский Вацлав Леопольдович   :   329
Попов Иван Иванович   :   323
Свентицкая Мария Хрисанфовна   :   313
Адрианов Александр Васильевич   :   305
Анучин Дмитрий Николаевич   :   295
Гуркин Григорий Иванович   :   244
Лаврский Валериан Викторович   :   230
Козьмин Николай Николаевич   :   228
Лаврский Константин Викторович   :   225
Костюрина Мария Николаевна   :   206
Руднев Андрей Дмитриевич   :   206
Педашенко-Третьякова Мария Ивановна   :   191
Красноженова Мария Васильевна   :  

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:01:37 epoch:    0, train/val loss = 8.01187 / 7.56190, acc =  59.957 /  63.240
00:03:10 epoch:    1, train/val loss = 7.49091 / 7.05529, acc =  62.168 /  64.898
00:04:43 epoch:    2, train/val loss = 7.11421 / 6.85029, acc =  64.266 /  65.798
00:06:17 epoch:    3, train/val loss = 6.84242 / 6.62715, acc =  65.997 /  67.575
00:07:50 epoch:    4, train/val loss = 6.59731 / 6.33457, acc =  67.304 /  68.640
00:09:26 epoch:    5, train/val loss = 6.50179 / 6.24444, acc =  68.005 /  69.043
00:11:03 epoch:    6, train/val loss = 6.30655 / 6.15240, acc =  69.048 /  69.659
00:12:35 epoch:    7, train/val loss = 6.29509 / 6.22044, acc =  68.910 /  69.588
00:14:11 epoch:    8, train/val loss = 6.22839 / 6.13122, acc =  69.401 /  70.322
00:15:47 epoch:    9, train/val loss = 6.13243 / 5.82928, acc =  69.851 /  71.246
00:17:23 epoch:   10, train/val loss = 6.11738 / 5.85826, acc =  70.324 /  71.127
00:19:01 epoch:   11, train/val loss = 5.99193 / 5.83046, acc =  70.270 /  72.027
00:20:33 epoch: 

...

train/val acc : 76.80229949951172 / 75.86451721191406
Siam model parameters number: 0
Class model parameters number: 563812
train dataset: 16687, test dataset: 4222
--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 880])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 100])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:00:30 epoch:    0, train/val loss = 3.51708 / 3.38036, acc =  13.669 /  15.751
00:01:01 epoch:    1, train/val loss = 3.18379 / 3.29042, acc =  18.781 /  15.514
00:01:32 epoch:    2, train/val loss = 3.09679 / 3.20805, acc =  20.453 /  18.025
00:02:03 epoch:    3, train/val loss = 3.02075 / 3.20030, acc =  21.310 /  18.404
00:02:33 epoch:    4, train/val loss = 2.95327 / 3.19398, acc =  23.114 /  19.375
00:03:04 epoch:    5, train/val loss = 2.94514 / 3.12217, acc =  23.240 /  20.369
00:03:35 epoch:    6, train/val loss = 2.89262 / 3.11347, acc =  24.169 /  19.209
00:04:06 epoch:    7, train/val loss = 2.85654 / 3.11929, acc =  24.186 /  19.351
00:04:37 epoch:    8, train/val loss = 2.82061 / 3.19208, acc =  25.355 /  20.085
00:05:08 epoch:    9, train/val loss = 2.81203 / 3.08039, acc =  25.211 /  20.891
00:05:39 epoch:   10, train/val loss = 2.76050 / 3.07371, acc =  26.188 /  21.743
00:06:10 epoch:   11, train/val loss = 2.74915 / 3.09423, acc =  26.050 /  21.198
00:06:40 epoch: 

In [39]:
config = {
    "device" : DEVICE,

    "type" : "strokes",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 205507

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
Мендельсон Николай Михайлович                        :   426 = 340 + 86
Серошевский Вацлав Леопольдович                      :   412 = 329 + 83
Попов Иван Иванович                                  : 

# Strokes 2

In [40]:
class NetworkStrokes2(nn.Module):
    def __init__(self, in_channels=3, out_channels=16, output_emb=1000, rnn_hidden_size=128, rnn_layers=1, dropout=0.3):
        super().__init__()
        
        # Свёрточный энкодер (извлекает визуальные признаки)
        channels = [in_channels, 4, 8, 16, out_channels]
        
        self.cnn = nn.Sequential(
            # [B, 1, H, W] -> [B, 64, H/2, W/2]
            nn.Conv2d(in_channels, channels[1], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels[1]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=(1, 2)),
            
            # [B, 64, H/2, W/2] -> [B, 128, H/4, W/4]
            nn.Conv2d(channels[1], channels[2], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels[2]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=(1, 2)),
            
            # [B, 128, H/4, W/4] -> [B, 256, H/8, W/8]
            nn.Conv2d(channels[2], channels[3], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels[3]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=(1, 2)),
            
            # [B, 256, H/8, W/8] -> [B, 512, H/16, W/16]
            nn.Conv2d(channels[3], channels[4], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels[4]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 2)),  # Сжимаем только по высоте
        )
        
        # Рекуррентная сеть (обрабатывает последовательность признаков)
        self.rnn = nn.LSTM(
            input_size=23 * out_channels,
            hidden_size=rnn_hidden_size,
            num_layers=rnn_layers,
            bidirectional=True,
            dropout=dropout if rnn_layers > 1 else 0,
            batch_first=True
        )
        
        # Проекция в эмбеддинг
        self.proj = nn.Linear(2 * rnn_hidden_size, output_emb)  # x2 из-за bidirectional
        
    def forward(self, x):
        # x: [B, 3, H, W]
        
        features = self.cnn(x)  # [B, 512, H', W']
        
        B, C, H, W = features.shape
        features = features.permute(0, 3, 2, 1)  # [B, W, H, C]
        features = features.reshape(B, W, H * C)  # [B, W, H*C]
        
        # print(f"{B=}, {C=}, {H=}, {W=}")
        # print(features.shape)
        rnn_out, _ = self.rnn(features)  # [B, W, 2*rnn_hidden_size] (bidirectional)
        
        # Усредняем по временной оси
        seq_embedding = torch.mean(rnn_out, dim=1)  # [B, 2*rnn_hidden_size]
        
        embedding = self.proj(seq_embedding)  # [B, emb_dim]
        
        # Нормализуем эмбеддинг (опционально)
        # embedding = F.normalize(embedding, p=2, dim=1)
        
        return embedding

In [189]:
for i in model.children():
    print(number_of_parameters(i))

3984
509952
257000


In [188]:
model = NetworkStrokes2().to(DEVICE)
print(f"Siam model parameters number: {number_of_parameters(model)}")


Siam model parameters number: 770936


In [169]:
model = NetworkStrokes2().to(DEVICE)
print(f"Siam model parameters number: {number_of_parameters(model)}")

Siam model parameters number: 981952


In [190]:
transform = transforms.Compose(
    [
            transforms.ToTensor(),
            Min_size(50),
    ])

dataset = My_TripletFolder("КККМFOLDER_strokes_bin_ok", transform, topk=3)

In [191]:
test_pipeline(model, dataset, device=DEVICE, batch_size=64,
                  collate_fn = pad_collate_siam, type = "siam")

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 702])
model:   torch.Size([64, 1000])
--- test_pipeline end



## 3 class

#### Unbalanced 

In [221]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

Dataset length: 2865
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
train dataset: 2292, test dataset: 573

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 741])
model:   torch.Size([64, 1000])
--- test_pipeline end

Make logs/2025-05-05 00:50:54 siam_strokes_Two3
Make info/2025-05-05 00:50:54 siam_strokes_Two3

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:10 epoch:    0, train/val loss = 7.11999 / 5.49821, acc =  67.016 /  72.949
00:00:19 epoch:    1, train/val loss = 5.12966 / 4.76227, acc =  75.567 /  78.883
00:00:28 epoch:    2, train/val loss = 4.04229 / 3.29958, acc =  81.326 /  84.817
00:00:38 epoch:    3, train/val loss = 2.97654 / 3.24179, acc =  86.213 /  85.864
00:00:47 epoch:    4, train/val loss = 2.53210 / 2.45487, acc =  88.307 /  90.576
00:00:56 epoch:    5, train/val loss = 2.03929 / 1.69693, acc =  91.099 /  92.496
00:01:06 epoch:    6, train/val loss = 1.76325 / 2.35900, acc =  92.801 /  89.180
00:01:15 epoch:    7, train/val loss = 1.70346 / 1.82783, acc =  92.365 /  91.274
00:01:24 epoch:    8, train/val loss = 1.50451 / 1.48652, acc =  93.368 /  94.241
00:01:34 epoch:    9, train/val loss = 1.23448 / 0.96238, acc =  94.983 /  95.812
00:01:43 epoch:   10, train/val loss = 0.94117 / 0.96039, acc =  96.030 /  96.161
00:01:52 epoch:   11, train/val loss = 0.79202 / 1.05308, acc =  96.902 /  95.986
00:02:01 epoch: 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:02 epoch:    0, train/val loss = 0.14613 / 0.10763, acc =  95.593 /  97.382
00:00:05 epoch:    1, train/val loss = 0.07608 / 0.09344, acc =  97.731 /  97.382
00:00:08 epoch:    2, train/val loss = 0.06779 / 0.09821, acc =  97.862 /  97.208
00:00:11 epoch:    3, train/val loss = 0.06625 / 0.09171, acc =  97.862 /  97.557
00:00:14 epoch:    4, train/val loss = 0.07310 / 0.09125, acc =  97.688 /  97.906
00:00:16 epoch:    5, train/val loss = 0.06652 / 0.10472, acc =  97.949 /  97.382
00:00:19 epoch:    6, train/val loss = 0.06877 / 0.09461, acc =  97.688 /  97.557
00:00:22 epoch:    7, train/val loss = 0.06333 / 0.09935, acc =  97.993 /  97.557
00:00:25 epoch:    8, train/val loss = 0.06423 / 0.09749, acc =  97.862 /  97.557
00:00:28 epoch:    9, train/val loss = 0.06331 / 0.09271, acc =  97.688 /  97.731
00:00:30 epoch:   10, train/val loss = 0.06172 / 0.09663, acc =  97.993 /  97.557
00:00:33 epoch:   11, train/val loss = 0.06631 / 0.09198, acc =  98.080 /  97.731
00:00:36 epoch: 

In [41]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936



Dataset length: 2865
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
train dataset: 2292, test dataset: 573

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 728])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-05 00:50:54 siam_strokes_Two3
load_model: logs/2025-05-05 00:50:54 siam_strokes_Two3/siam/13 epoch.pt
train/val acc : 96.98953247070312 / 97.55672454833984
Siam model parameters number: 0
Class model parameters number: 514051
Dataset length: 2865
train dataset: 2292, test dataset: 573

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 719])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

load_model: logs/2025-05-05 00:50:54 siam_strokes_Two3/class/24 epoch.pt
train/val acc : 97.81849980354309 / 99.30192232131958
hits@k: {1: 0.9781849912739965, 2: 0.997818499127399

#### Balanced

In [223]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
train dataset: 2291, test dataset: 574

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 739])
model:   torch.Size([64, 1000])
--- test_pipeline end

Make logs/2025-05-05 01:00:10 siam_strokes_Two3_balanced
Make info/2025-05-05 01:00:10 siam_strokes_Two3_balanced

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:11 epoch:    0, train/val loss = 6.88158 / 5.05360, acc =  68.311 /  75.784
00:00:21 epoch:    1, train/val loss = 3.48526 / 3.34106, acc =  84.243 /  84.669
00:00:30 epoch:    2, train/val loss = 2.04969 / 2.57250, acc =  91.052 /  87.631
00:00:39 epoch:    3, train/val loss = 1.89925 / 2.68805, acc =  91.838 /  86.934
00:00:48 epoch:    4, train/val loss = 1.58016 / 2.26463, acc =  93.147 /  89.547
00:00:58 epoch:    5, train/val loss = 1.36560 / 1.70618, acc =  94.369 /  93.380
00:01:07 epoch:    6, train/val loss = 1.29498 / 1.63537, acc =  94.544 /  93.031
00:01:16 epoch:    7, train/val loss = 1.04735 / 1.85983, acc =  95.417 /  91.638
00:01:25 epoch:    8, train/val loss = 0.87932 / 1.31062, acc =  96.421 /  94.948
00:01:35 epoch:    9, train/val loss = 0.85734 / 1.54756, acc =  96.421 /  92.857
00:01:44 epoch:   10, train/val loss = 0.74588 / 1.86713, acc =  96.901 /  92.334
00:01:53 epoch:   11, train/val loss = 0.71691 / 2.03583, acc =  97.163 /  91.463
00:02:02 epoch: 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:02 epoch:    0, train/val loss = 0.11814 / 0.03573, acc =  95.722 /  99.129
00:00:05 epoch:    1, train/val loss = 0.07898 / 0.06002, acc =  97.948 /  98.955
00:00:08 epoch:    2, train/val loss = 0.04620 / 0.01947, acc =  98.560 /  99.129
00:00:11 epoch:    3, train/val loss = 0.07561 / 0.02688, acc =  97.599 /  99.477
00:00:14 epoch:    4, train/val loss = 0.04640 / 0.02471, acc =  98.560 /  99.129
00:00:17 epoch:    5, train/val loss = 0.05853 / 0.01362, acc =  98.123 /  99.652
00:00:19 epoch:    6, train/val loss = 0.03913 / 0.05110, acc =  98.734 /  98.780
00:00:22 epoch:    7, train/val loss = 0.05187 / 0.02490, acc =  98.429 /  99.303
00:00:25 epoch:    8, train/val loss = 0.05640 / 0.03871, acc =  98.079 /  99.303
00:00:28 epoch:    9, train/val loss = 0.04739 / 0.02344, acc =  98.167 /  99.129
00:00:31 epoch:   10, train/val loss = 0.04886 / 0.02112, acc =  98.298 /  99.477
00:00:34 epoch:   11, train/val loss = 0.04775 / 0.02876, acc =  98.254 /  99.652
00:00:36 epoch: 

In [42]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
train dataset: 2291, test dataset: 574

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 938])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-05 01:00:10 siam_strokes_Two3_balanced
load_model: logs/2025-05-05 01:00:10 siam_strokes_Two3_balanced/siam/27 epoch.pt
train/val acc : 98.51593017578125 / 98.60626983642578
Siam model parameters number: 0
Class model parameters number: 514051
train dataset: 2291, test dataset: 574

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 778])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

load_model: logs/2025-05-05 01:00:10 siam_strokes_Two3_balanced/class/17 epoch.pt
train/val acc

### 10 class

#### Unbalanced 

In [43]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

Dataset length: 6670
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
train dataset: 5336, test dataset: 1334

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 731])
model:   torch.Size([64, 1000])
--- test_pipeline end

Make logs/2025-05-05 01:53:11 siam_strokes_Two10
Make info/2025-05-05 01:53:11 siam_strokes_Two10

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:35 epoch:    0, train/val loss = 7.82335 / 6.98763, acc =  60.776 /  64.168
00:01:06 epoch:    1, train/val loss = 6.79731 / 6.64822, acc =  65.648 /  66.717
00:01:35 epoch:    2, train/val loss = 6.65182 / 6.29520, acc =  66.867 /  70.690
00:02:06 epoch:    3, train/val loss = 6.30232 / 6.60192, acc =  68.797 /  66.342
00:02:36 epoch:    4, train/val loss = 5.86665 / 5.88138, acc =  71.627 /  71.664
00:03:05 epoch:    5, train/val loss = 5.39426 / 5.03417, acc =  73.445 /  75.487
00:03:35 epoch:    6, train/val loss = 5.10163 / 4.72497, acc =  75.750 /  77.736
00:04:06 epoch:    7, train/val loss = 4.91502 / 4.42017, acc =  76.237 /  78.861
00:04:37 epoch:    8, train/val loss = 4.46398 / 4.18515, acc =  78.261 /  79.760
00:05:07 epoch:    9, train/val loss = 4.29760 / 4.24825, acc =  79.816 /  78.786
00:05:38 epoch:   10, train/val loss = 4.09492 / 4.08012, acc =  80.847 /  79.985
00:06:09 epoch:   11, train/val loss = 3.92317 / 3.95170, acc =  81.559 /  80.885
00:06:40 epoch: 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:09 epoch:    0, train/val loss = 0.78624 / 0.62682, acc =  73.482 /  79.010
00:00:18 epoch:    1, train/val loss = 0.62193 / 0.57436, acc =  78.280 /  79.910
00:00:26 epoch:    2, train/val loss = 0.59789 / 0.57953, acc =  79.310 /  79.160
00:00:35 epoch:    3, train/val loss = 0.58105 / 0.57277, acc =  79.835 /  79.460
00:00:44 epoch:    4, train/val loss = 0.56474 / 0.54279, acc =  80.847 /  80.510
00:00:53 epoch:    5, train/val loss = 0.55207 / 0.56390, acc =  80.903 /  79.910
00:01:04 epoch:    6, train/val loss = 0.54485 / 0.54500, acc =  81.615 /  80.285
00:01:14 epoch:    7, train/val loss = 0.53377 / 0.56031, acc =  81.391 /  79.835
00:01:25 epoch:    8, train/val loss = 0.53034 / 0.55255, acc =  81.690 /  80.360
00:01:36 epoch:    9, train/val loss = 0.52547 / 0.53168, acc =  82.046 /  81.334
00:01:46 epoch:   10, train/val loss = 0.51232 / 0.51164, acc =  82.478 /  81.184
00:01:56 epoch:   11, train/val loss = 0.51031 / 0.53965, acc =  82.515 /  81.034
00:02:07 epoch: 

In [44]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

Dataset length: 6670
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
train dataset: 5336, test dataset: 1334

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 1088])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-05 01:53:11 siam_strokes_Two10
load_model: logs/2025-05-05 01:53:11 siam_strokes_Two10/siam/25 epoch.pt
train/val acc : 90.723388671875 / 89.9550247192

#### Balanced

In [45]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
train dataset: 5333, test dataset: 1337

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 1050])
model:   torch.Size([64, 1000])
--- test_pipeline end

Make logs/2025-05-05 02:15:00 siam_strokes_Two

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:30 epoch:    0, train/val loss = 7.98743 / 7.36066, acc =  60.829 /  62.977
00:01:00 epoch:    1, train/val loss = 7.01815 / 6.97012, acc =  64.167 /  65.071
00:01:30 epoch:    2, train/val loss = 6.03766 / 5.82476, acc =  70.767 /  71.952
00:02:01 epoch:    3, train/val loss = 5.35829 / 5.30269, acc =  74.180 /  75.168
00:02:33 epoch:    4, train/val loss = 5.01757 / 4.81242, acc =  76.261 /  75.991
00:03:07 epoch:    5, train/val loss = 4.79178 / 5.02587, acc =  77.180 /  75.991
00:03:39 epoch:    6, train/val loss = 4.68282 / 4.80250, acc =  78.080 /  77.562
00:04:12 epoch:    7, train/val loss = 4.59247 / 4.91276, acc =  78.399 /  75.916
00:04:43 epoch:    8, train/val loss = 4.20713 / 4.78962, acc =  80.086 /  76.889
00:05:15 epoch:    9, train/val loss = 3.95139 / 4.14291, acc =  81.624 /  80.628
00:05:46 epoch:   10, train/val loss = 4.11542 / 4.79394, acc =  80.480 /  77.263
00:06:16 epoch:   11, train/val loss = 3.81399 / 4.40688, acc =  82.486 /  80.105
00:06:47 epoch: 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:08 epoch:    0, train/val loss = 0.70953 / 0.57632, acc =  77.442 /  82.199
00:00:16 epoch:    1, train/val loss = 0.58497 / 0.53019, acc =  80.705 /  83.620
00:00:25 epoch:    2, train/val loss = 0.56714 / 0.53915, acc =  80.574 /  83.844
00:00:33 epoch:    3, train/val loss = 0.54079 / 0.51270, acc =  82.149 /  84.742
00:00:41 epoch:    4, train/val loss = 0.52461 / 0.56159, acc =  82.955 /  82.199
00:00:50 epoch:    5, train/val loss = 0.51465 / 0.51379, acc =  82.730 /  82.498
00:00:58 epoch:    6, train/val loss = 0.52372 / 0.49188, acc =  82.449 /  85.116
00:01:06 epoch:    7, train/val loss = 0.49693 / 0.53805, acc =  83.236 /  80.778
00:01:15 epoch:    8, train/val loss = 0.51934 / 0.44223, acc =  82.336 /  86.911
00:01:23 epoch:    9, train/val loss = 0.49184 / 0.54243, acc =  83.311 /  83.171
00:01:31 epoch:   10, train/val loss = 0.49548 / 0.49160, acc =  83.611 /  83.321
00:01:39 epoch:   11, train/val loss = 0.48513 / 0.54158, acc =  83.349 /  83.246
00:01:48 epoch: 

In [46]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
train dataset: 5333, test dataset: 1337

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 904])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-05 02:15:00 siam_strokes_Two10_bal

### 20 class

#### Unbalanced 

In [47]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

Dataset length: 10216
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
Мендельсон Николай Михайлович                        :   426
Серошевский Вацлав Леопольдович                      :   412
Попов Иван Иванович                                  :   404
Свентицкая Мария Хрисанфовна                         :   392
Адрианов Александр Васильевич                        :   382
Анучин Дмит

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

00:00:52 epoch:    0, train/val loss = 7.59634 / 6.53508, acc =  61.637 /  68.346
00:01:47 epoch:    1, train/val loss = 6.33700 / 5.82371, acc =  69.102 /  71.967
00:02:40 epoch:    2, train/val loss = 5.91330 / 5.57747, acc =  71.268 /  73.826
00:03:26 epoch:    3, train/val loss = 5.67506 / 5.22805, acc =  72.394 /  74.217
00:04:11 epoch:    4, train/val loss = 5.25949 / 5.21425, acc =  75.037 /  75.391
00:05:05 epoch:    5, train/val loss = 5.07579 / 5.09564, acc =  76.358 /  75.147
00:06:00 epoch:    6, train/val loss = 4.98850 / 4.93726, acc =  76.114 /  76.321
00:06:56 epoch:    7, train/val loss = 4.85644 / 4.57862, acc =  77.166 /  78.963
00:07:51 epoch:    8, train/val loss = 4.54740 / 4.54235, acc =  78.488 /  78.082
00:08:44 epoch:    9, train/val loss = 4.42105 / 4.63958, acc =  79.760 /  77.789
00:09:31 epoch:   10, train/val loss = 4.28666 / 4.22435, acc =  80.078 /  80.039
00:10:21 epoch:   11, train/val loss = 4.06208 / 3.91072, acc =  80.543 /  82.094
00:11:15 epoch: 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

00:00:14 epoch:    0, train/val loss = 1.21924 / 1.04394, acc =  60.365 /  64.775
00:00:29 epoch:    1, train/val loss = 1.03493 / 1.03040, acc =  65.088 /  65.900
00:00:43 epoch:    2, train/val loss = 1.00610 / 1.01624, acc =  65.786 /  65.313
00:00:58 epoch:    3, train/val loss = 0.97102 / 0.99794, acc =  67.462 /  66.928
00:01:13 epoch:    4, train/val loss = 0.96527 / 0.96617, acc =  67.670 /  68.297
00:01:27 epoch:    5, train/val loss = 0.94630 / 0.97556, acc =  67.890 /  67.710
00:01:46 epoch:    6, train/val loss = 0.93167 / 0.99210, acc =  68.771 /  67.172
00:02:04 epoch:    7, train/val loss = 0.92844 / 0.96578, acc =  68.882 /  67.270
00:02:22 epoch:    8, train/val loss = 0.92016 / 0.95837, acc =  68.612 /  68.591
00:02:40 epoch:    9, train/val loss = 0.90330 / 0.92928, acc =  69.261 /  69.080
00:02:59 epoch:   10, train/val loss = 0.89232 / 0.96168, acc =  69.714 /  67.808
00:03:19 epoch:   11, train/val loss = 0.88644 / 0.96058, acc =  70.020 /  68.249
00:03:36 epoch: 

In [48]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

Dataset length: 10216
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
Мендельсон Николай Михайлович                        :   426
Серошевский Вацлав Леопольдович                      :   412
Попов Иван Иванович                                  :   404
Свентицкая Мария Хрисанфовна                         :   392
Адрианов Александр Васильевич                        :   382
Анучин Дмит

#### Balanced

In [49]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
Мендельсон Николай Михайлович                        :   426 = 340 + 86
Серошевский Вацлав Леопольдович                      :   412 = 329 + 83
Попов Иван Иванович                                  : 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

00:00:47 epoch:    0, train/val loss = 7.64835 / 7.06551, acc =  61.494 /  65.383
00:01:38 epoch:    1, train/val loss = 7.02505 / 6.18198, acc =  64.911 /  68.991
00:02:32 epoch:    2, train/val loss = 6.21858 / 5.91538, acc =  69.712 /  72.989
00:03:26 epoch:    3, train/val loss = 5.48834 / 5.54670, acc =  73.313 /  74.939
00:04:19 epoch:    4, train/val loss = 5.24060 / 5.03517, acc =  75.358 /  76.499
00:05:13 epoch:    5, train/val loss = 4.92227 / 4.64594, acc =  76.632 /  77.767
00:06:06 epoch:    6, train/val loss = 4.72748 / 4.52718, acc =  77.955 /  78.157
00:07:00 epoch:    7, train/val loss = 4.61199 / 4.70142, acc =  78.065 /  77.474
00:07:57 epoch:    8, train/val loss = 4.52852 / 4.10030, acc =  78.971 /  79.620
00:08:49 epoch:    9, train/val loss = 4.34344 / 4.28357, acc =  80.012 /  79.815
00:09:43 epoch:   10, train/val loss = 4.10019 / 4.13824, acc =  81.139 /  81.424
00:10:34 epoch:   11, train/val loss = 4.01896 / 4.21692, acc =  80.919 /  80.546
00:11:28 epoch: 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

00:00:14 epoch:    0, train/val loss = 1.45038 / 1.31858, acc =  51.990 /  56.899
00:00:28 epoch:    1, train/val loss = 1.22478 / 1.19698, acc =  59.057 /  61.287
00:00:43 epoch:    2, train/val loss = 1.17859 / 1.23196, acc =  61.225 /  60.166
00:00:58 epoch:    3, train/val loss = 1.13399 / 1.15919, acc =  61.764 /  61.824
00:01:12 epoch:    4, train/val loss = 1.11089 / 1.20760, acc =  62.658 /  60.946
00:01:27 epoch:    5, train/val loss = 1.09243 / 1.13863, acc =  63.331 /  61.726
00:01:42 epoch:    6, train/val loss = 1.09894 / 1.14841, acc =  63.478 /  61.043
00:01:59 epoch:    7, train/val loss = 1.08169 / 1.07593, acc =  64.666 /  62.409
00:02:21 epoch:    8, train/val loss = 1.05255 / 1.06393, acc =  64.801 /  65.724
00:02:41 epoch:    9, train/val loss = 1.04747 / 1.10929, acc =  64.911 /  65.090
00:02:58 epoch:   10, train/val loss = 1.03960 / 1.01460, acc =  65.805 /  66.309
00:03:15 epoch:   11, train/val loss = 1.01327 / 1.11487, acc =  66.258 /  64.115
00:03:31 epoch: 

In [50]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
Мендельсон Николай Михайлович                        :   426 = 340 + 86
Серошевский Вацлав Леопольдович                      :   412 = 329 + 83
Попов Иван Иванович                                  : 

### 100 class

#### Unbalanced 

In [51]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

Dataset length: 20909
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
Мендельсон Николай Михайлович                        :   426
Серошевский Вацлав Леопольдович                      :   412
Попов Иван Иванович                                  :   404
Свентицкая Мария Хрисанфовна                         :   392
Адрианов Александр Васильевич                        :   382
Анучин Дмит

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:01:43 epoch:    0, train/val loss = 7.82096 / 7.33637, acc =  60.399 /  62.865
00:03:38 epoch:    1, train/val loss = 6.71485 / 6.09293, acc =  66.958 /  70.851
00:05:31 epoch:    2, train/val loss = 5.99092 / 5.91392, acc =  71.041 /  71.234
00:07:20 epoch:    3, train/val loss = 5.60673 / 5.46124, acc =  73.073 /  73.410
00:09:21 epoch:    4, train/val loss = 5.21222 / 5.08399, acc =  74.951 /  75.753
00:11:21 epoch:    5, train/val loss = 4.96729 / 4.95934, acc =  76.272 /  76.877
00:13:19 epoch:    6, train/val loss = 4.67512 / 4.84100, acc =  77.904 /  77.403
00:15:11 epoch:    7, train/val loss = 4.56974 / 4.57300, acc =  78.382 /  78.647
00:17:04 epoch:    8, train/val loss = 4.42109 / 4.31180, acc =  79.279 /  79.699
00:19:05 epoch:    9, train/val loss = 4.30174 / 4.29355, acc =  80.074 /  79.579
00:21:05 epoch:   10, train/val loss = 4.26785 / 4.17955, acc =  80.200 /  79.794
00:23:06 epoch:   11, train/val loss = 4.10067 / 4.02463, acc =  80.983 /  81.277
00:25:05 epoch: 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:00:34 epoch:    0, train/val loss = 2.49455 / 2.15449, acc =  37.861 /  43.209
00:01:08 epoch:    1, train/val loss = 2.10626 / 2.11350, acc =  43.606 /  42.970
00:01:41 epoch:    2, train/val loss = 2.02602 / 2.04000, acc =  44.933 /  45.863
00:02:14 epoch:    3, train/val loss = 1.95804 / 2.00642, acc =  45.908 /  46.222
00:02:47 epoch:    4, train/val loss = 1.91590 / 1.98689, acc =  47.026 /  46.341
00:03:19 epoch:    5, train/val loss = 1.88271 / 1.95911, acc =  47.773 /  47.728
00:03:51 epoch:    6, train/val loss = 1.84243 / 1.95785, acc =  48.664 /  47.059
00:04:23 epoch:    7, train/val loss = 1.81690 / 1.93343, acc =  48.897 /  48.063
00:04:55 epoch:    8, train/val loss = 1.78779 / 1.94785, acc =  49.650 /  47.561
00:05:29 epoch:    9, train/val loss = 1.76398 / 1.93604, acc =  50.105 /  48.254
00:06:01 epoch:   10, train/val loss = 1.73666 / 1.91733, acc =  50.499 /  48.063
00:06:33 epoch:   11, train/val loss = 1.70822 / 1.90429, acc =  50.942 /  48.757
00:07:05 epoch: 

In [52]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

Dataset length: 20909
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
Мендельсон Николай Михайлович                        :   426
Серошевский Вацлав Леопольдович                      :   412
Попов Иван Иванович                                  :   404
Свентицкая Мария Хрисанфовна                         :   392
Адрианов Александр Васильевич                        :   382
Анучин Дмит

#### Balanced

In [53]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
Мендельсон Николай Михайлович                        :   426 = 340 + 86
Серошевский Вацлав Леопольдович                      :   412 = 329 + 83
Попов Иван Иванович                                  : 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:01:40 epoch:    0, train/val loss = 7.38331 / 6.60404, acc =  63.654 /  67.290
00:03:18 epoch:    1, train/val loss = 6.36896 / 5.86039, acc =  68.802 /  71.743
00:04:58 epoch:    2, train/val loss = 5.77014 / 5.60436, acc =  72.368 /  72.572
00:06:37 epoch:    3, train/val loss = 5.65027 / 5.14992, acc =  72.494 /  75.557
00:08:17 epoch:    4, train/val loss = 5.30639 / 5.16633, acc =  74.507 /  75.107
00:09:56 epoch:    5, train/val loss = 5.10465 / 4.93827, acc =  75.370 /  76.551
00:11:34 epoch:    6, train/val loss = 4.93722 / 4.62734, acc =  76.599 /  78.612
00:13:09 epoch:    7, train/val loss = 4.66476 / 4.67524, acc =  78.169 /  77.096
00:14:42 epoch:    8, train/val loss = 4.51270 / 4.49253, acc =  78.906 /  79.228
00:16:18 epoch:    9, train/val loss = 4.45179 / 4.31043, acc =  78.840 /  80.152
00:17:53 epoch:   10, train/val loss = 4.27689 / 4.33255, acc =  80.008 /  79.346
00:19:28 epoch:   11, train/val loss = 4.16272 / 4.15980, acc =  80.164 /  80.625
00:21:03 epoch: 

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:00:31 epoch:    0, train/val loss = 2.75673 / 2.43679, acc =  28.214 /  32.875
00:01:02 epoch:    1, train/val loss = 2.34514 / 2.29385, acc =  35.441 /  35.978
00:01:34 epoch:    2, train/val loss = 2.25713 / 2.29915, acc =  37.670 /  37.068
00:02:05 epoch:    3, train/val loss = 2.15993 / 2.25369, acc =  39.468 /  38.489
00:02:36 epoch:    4, train/val loss = 2.09853 / 2.23581, acc =  40.559 /  39.626
00:03:08 epoch:    5, train/val loss = 2.04161 / 2.22518, acc =  41.967 /  39.744
00:03:40 epoch:    6, train/val loss = 1.97883 / 2.20657, acc =  43.854 /  39.886
00:04:12 epoch:    7, train/val loss = 1.95611 / 2.20380, acc =  44.040 /  40.076
00:04:44 epoch:    8, train/val loss = 1.89071 / 2.20186, acc =  45.173 /  40.597
00:05:16 epoch:    9, train/val loss = 1.88756 / 2.20241, acc =  46.144 /  41.473
00:05:47 epoch:   10, train/val loss = 1.86690 / 2.15725, acc =  46.066 /  40.857
00:06:19 epoch:   11, train/val loss = 1.81732 / 2.23164, acc =  47.204 /  40.597
00:06:51 epoch: 

In [54]:
config = {
    "device" : DEVICE,

    "type" : "strokes_Two",

    "model" : "model2",
    "siam_emb_dim" : 1000,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam model parameters number: 770936

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
Мендельсон Николай Михайлович                        :   426 = 340 + 86
Серошевский Вацлав Леопольдович                      :   412 = 329 + 83
Попов Иван Иванович                                  : 

## Strokes ResNet

### 3 class

#### Unbalanced 

In [61]:
ls

info/     logs/     КККМFOLDER_bin/        КККМFOLDER_strokes_bin_ok/
info.tgz  logs.tgz  КККМFOLDER_bin.tar.xz  КККМFOLDER_strokes_bin_ok.tar.xz


In [226]:
exp = "2025-05-05 01:06:45 siam_strokes_Two10"

!rm -r "logs/{exp}"
!rm -r "info/{exp}"

In [69]:
file = "Total_Main.txt"

!rm "{file}"

folder = "info"
for exp in sorted(os.listdir(folder)):
    if exp[0] == ".":
        continue


    write2file(file, f"{exp}\n")
    write2file(file, f" _____{exp.split()[-1].center(76).upper()}_____ \n")

    text = open(os.path.join(folder, exp, "Main.txt")).read()

    write2file(file, text)

In [59]:
exp1 = "2025-05-04 12:13:07 siam_strokes100_balanced"
exp2 = "2025-05-03 23:58:08 siam_strokes100_balanced"

!mv "logs/{exp1}" "logs/{exp2}"
!mv "info/{exp1}" "info/{exp2}"

In [60]:
!tar -cvzf logs.tgz logs/
!tar -cvzf info.tgz info/

logs/
logs/2025-05-03 18:47:02 siam_window3/
logs/2025-05-03 18:47:02 siam_window3/siam/
logs/2025-05-03 18:47:02 siam_window3/siam/0 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/logs.pickle
logs/2025-05-03 18:47:02 siam_window3/siam/1 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/2 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/3 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/4 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/5 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/6 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/7 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/8 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/9 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/10 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/11 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/12 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/13 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/siam/14 epoch.pt
logs/2025-05-03 18:47:02 siam_window3/sia

In [21]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

Dataset length: 2865
Потанина Александра Викторовна   :   1384
Фарафонтова Таисия Михайловна   :   775
Вагнер (Дзвонкевич) Екатерина Николаевна   :   706

train dataset: 2292, test dataset: 573
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 725])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-04 08:28:02 siam_strokes_ResNet3
Make info/2025-05-04 08:28:02 siam_strokes_ResNet3

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:15 epoch:    0, train/val loss = 6.40760 / 5.35425, acc =  76.920 /  76.789
00:00:25 epoch:    1, train/val loss = 4.68819 / 4.72779, acc =  79.668 /  77.487
00:00:34 epoch:    2, train/val loss = 3.88783 / 3.49945, acc =  82.417 /  85.864
00:00:43 epoch:    3, train/val loss = 3.63496 / 3.37501, acc =  84.337 /  85.166
00:00:51 epoch:    4, train/val loss = 3.49737 / 3.62563, acc =  83.770 /  85.515
00:01:00 epoch:    5, train/val loss = 3.25171 / 3.42471, acc =  85.820 /  83.421
00:01:09 epoch:    6, train/val loss = 3.30416 / 3.46649, acc =  86.475 /  84.642
00:01:17 epoch:    7, train/val loss = 3.13884 / 2.93962, acc =  85.602 /  86.736
00:01:26 epoch:    8, train/val loss = 3.07709 / 3.34348, acc =  85.908 /  84.642
00:01:35 epoch:    9, train/val loss = 3.15345 / 3.28181, acc =  86.257 /  84.293
00:01:43 epoch:   10, train/val loss = 2.96840 / 3.13756, acc =  86.867 /  87.086
00:01:52 epoch:   11, train/val loss = 2.99234 / 3.63326, acc =  86.911 /  82.548
00:02:01 epoch: 

...

train/val acc : 88.87434387207031 / 87.95811462402344
Siam model parameters number: 0
Class model parameters number: 514051
Dataset length: 2865
train dataset: 2292, test dataset: 573

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 1021])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:02 epoch:    0, train/val loss = 0.27459 / 0.27949, acc =  89.354 /  89.878
00:00:05 epoch:    1, train/val loss = 0.22051 / 0.26768, acc =  91.623 /  90.052
00:00:08 epoch:    2, train/val loss = 0.21570 / 0.26648, acc =  92.016 /  90.576
00:00:10 epoch:    3, train/val loss = 0.20402 / 0.30050, acc =  92.234 /  89.180
00:00:13 epoch:    4, train/val loss = 0.20581 / 0.28370, acc =  92.539 /  89.878
00:00:16 epoch:    5, train/val loss = 0.19477 / 0.25818, acc =  93.368 /  90.576
00:00:19 epoch:    6, train/val loss = 0.19449 / 0.26154, acc =  92.496 /  90.401
00:00:21 epoch:    7, train/val loss = 0.18370 / 0.26418, acc =  93.543 /  90.750
00:00:24 epoch:    8, train/val loss = 0.17698 / 0.27726, acc =  93.412 /  89.878
00:00:27 epoch:    9, train/val loss = 0.17638 / 0.28090, acc =  93.630 /  89.703
00:00:29 epoch:   10, train/val loss = 0.17545 / 0.26383, acc =  93.717 /  89.878
00:00:32 epoch:   11, train/val loss = 0.17035 / 0.25857, acc =  93.586 /  90.750
00:00:35 epoch: 

In [52]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

Dataset length: 2865
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
train dataset: 2292, test dataset: 573

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 767])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-04 08:28:02 siam_strokes_ResNet3
load_model: logs/2025-05-04 08:28:02 siam_strokes_ResNet3/siam/27 epoch.pt
train/val acc : 88.13263702392578 / 87.08551788330078
Siam model parameters number: 0
Class model parameters number: 514051
Dataset length: 2865
train dataset: 2292, test dataset: 573

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 960])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

load_model: logs/2025-05-04 08:28:02 siam_strokes_ResNet3/class/24 epoch.pt
train/val acc : 94.72076892852783 / 96.8

#### Balanced

In [22]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

train dataset
Потанина Александра Викторовна   :   1107
Фарафонтова Таисия Михайловна   :   620
Вагнер (Дзвонкевич) Екатерина Николаевна   :   564

test dataset
Потанина Александра Викторовна   :   277
Фарафонтова Таисия Михайловна   :   155
Вагнер (Дзвонкевич) Екатерина Николаевна   :   142

train dataset: 2291, test dataset: 574
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 1041])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-04 08:34:05 siam_strokes_ResNet3_balanced
Make info/2025-05-04 08:34:05 siam_strokes_ResNet3_balanced

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:08 epoch:    0, train/val loss = 6.23477 / 4.98259, acc =  76.037 /  78.920
00:00:17 epoch:    1, train/val loss = 4.13084 / 3.79151, acc =  82.235 /  82.927
00:00:25 epoch:    2, train/val loss = 3.24347 / 3.33708, acc =  86.512 /  85.540
00:00:34 epoch:    3, train/val loss = 3.12854 / 2.98290, acc =  87.342 /  86.760
00:00:43 epoch:    4, train/val loss = 3.03067 / 2.97419, acc =  87.167 /  85.192
00:00:52 epoch:    5, train/val loss = 3.15813 / 3.17196, acc =  85.552 /  85.017
00:01:00 epoch:    6, train/val loss = 2.99110 / 3.73748, acc =  86.949 /  82.753
00:01:09 epoch:    7, train/val loss = 2.82134 / 2.71950, acc =  86.993 /  88.850
00:01:18 epoch:    8, train/val loss = 2.54451 / 3.25608, acc =  88.957 /  86.063
00:01:27 epoch:    9, train/val loss = 2.83943 / 2.87888, acc =  87.473 /  87.805
00:01:35 epoch:   10, train/val loss = 2.61031 / 3.18958, acc =  89.306 /  87.456
00:01:44 epoch:   11, train/val loss = 2.67149 / 2.71609, acc =  88.171 /  88.502
00:01:53 epoch: 

...

train/val acc : 88.86949157714844 / 86.06271362304688
Siam model parameters number: 0
Class model parameters number: 514051
train dataset: 2291, test dataset: 574
--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 1041])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

00:00:02 epoch:    0, train/val loss = 0.33758 / 0.23633, acc =  88.346 /  91.463
00:00:05 epoch:    1, train/val loss = 0.27584 / 0.17679, acc =  90.528 /  93.728
00:00:08 epoch:    2, train/val loss = 0.23116 / 0.18757, acc =  91.270 /  94.948
00:00:10 epoch:    3, train/val loss = 0.23361 / 0.26676, acc =  92.012 /  90.941
00:00:13 epoch:    4, train/val loss = 0.21611 / 0.21030, acc =  92.274 /  92.857
00:00:16 epoch:    5, train/val loss = 0.23384 / 0.17187, acc =  91.532 /  93.206
00:00:18 epoch:    6, train/val loss = 0.21507 / 0.19313, acc =  92.230 /  94.077
00:00:21 epoch:    7, train/val loss = 0.18250 / 0.17727, acc =  93.234 /  93.380
00:00:24 epoch:    8, train/val loss = 0.20806 / 0.25555, acc =  92.580 /  90.592
00:00:27 epoch:    9, train/val loss = 0.18541 / 0.19827, acc =  93.409 /  92.683
00:00:29 epoch:   10, train/val loss = 0.20611 / 0.20472, acc =  92.361 /  92.160
00:00:32 epoch:   11, train/val loss = 0.18458 / 0.20567, acc =  93.278 /  91.463
00:00:35 epoch: 

In [53]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 3,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
train dataset: 2291, test dataset: 574

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 1021])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-04 08:34:05 siam_strokes_ResNet3_balanced
load_model: logs/2025-05-04 08:34:05 siam_strokes_ResNet3_balanced/siam/16 epoch.pt
train/val acc : 88.60758972167969 / 89.54703521728516
Siam model parameters number: 0
Class model parameters number: 514051
train dataset: 2291, test dataset: 574

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 840])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 3])
--- test_pipeline end

load_model: logs/2025-05-04 08:34:05 siam_strokes_ResNet3_balanced/class/24 epoch

### 10 class

#### Unbalanced 

In [23]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

Dataset length: 6670
Потанина Александра Викторовна   :   1384
Фарафонтова Таисия Михайловна   :   775
Вагнер (Дзвонкевич) Екатерина Николаевна   :   706
Крутовский Владимир Михайлович   :   678
Ивановский Алексей Осипович   :   664
Антонова Валентина Константиновна   :   544
Семидалов Вениамин Иванович   :   518
Проскурякова Е. Павловна   :   489
Крутовская Лидия Симоновна   :   462
Ржавская Надежда Федоровна   :   450

train dataset: 5336, test dataset: 1334
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 995])
model:   torch.Size([64, 1000])
--- test_pipeline end
Make logs/2025-05-04 08:40:04 siam_strokes_ResNet10
Make info/2025-05-04 08:40:04 siam_strokes_ResNet10

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:32 epoch:    0, train/val loss = 7.63641 / 6.86466, acc =  64.318 /  65.742
00:00:55 epoch:    1, train/val loss = 6.40430 / 6.29423, acc =  69.097 /  70.240
00:01:19 epoch:    2, train/val loss = 6.01222 / 5.86785, acc =  70.652 /  73.238
00:01:43 epoch:    3, train/val loss = 5.89567 / 5.84545, acc =  71.945 /  71.814
00:02:06 epoch:    4, train/val loss = 5.53501 / 5.42913, acc =  73.538 /  73.013
00:02:30 epoch:    5, train/val loss = 5.61403 / 5.65372, acc =  73.163 /  71.814
00:02:53 epoch:    6, train/val loss = 5.20912 / 5.41681, acc =  75.562 /  74.363
00:03:17 epoch:    7, train/val loss = 5.19474 / 5.58167, acc =  76.068 /  73.313
00:03:40 epoch:    8, train/val loss = 5.10668 / 5.36144, acc =  75.712 /  74.138
00:04:03 epoch:    9, train/val loss = 5.21822 / 5.46693, acc =  75.487 /  73.163
00:04:27 epoch:   10, train/val loss = 5.04214 / 5.35663, acc =  75.656 /  74.213
00:04:50 epoch:   11, train/val loss = 4.90563 / 5.34101, acc =  76.555 /  74.588
00:05:13 epoch: 

...

train/val acc : 78.87931823730469 / 74.88755798339844
Siam model parameters number: 0
Class model parameters number: 517642
Dataset length: 6670
train dataset: 5336, test dataset: 1334

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 727])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 10])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:07 epoch:    0, train/val loss = 1.04104 / 0.89169, acc =  66.454 /  72.339
00:00:15 epoch:    1, train/val loss = 0.89449 / 0.85668, acc =  70.877 /  72.489
00:00:22 epoch:    2, train/val loss = 0.85839 / 0.85248, acc =  71.570 /  71.589
00:00:29 epoch:    3, train/val loss = 0.84621 / 0.84901, acc =  72.039 /  72.714
00:00:37 epoch:    4, train/val loss = 0.82870 / 0.84123, acc =  72.283 /  73.163
00:00:45 epoch:    5, train/val loss = 0.81585 / 0.86065, acc =  73.257 /  71.364
00:00:52 epoch:    6, train/val loss = 0.80122 / 0.84168, acc =  73.707 /  73.013
00:00:59 epoch:    7, train/val loss = 0.77926 / 0.85598, acc =  74.550 /  73.088
00:01:07 epoch:    8, train/val loss = 0.78685 / 0.83180, acc =  73.969 /  73.313
00:01:14 epoch:    9, train/val loss = 0.76073 / 0.82144, acc =  74.681 /  73.013
00:01:22 epoch:   10, train/val loss = 0.76051 / 0.84654, acc =  75.225 /  72.489
00:01:29 epoch:   11, train/val loss = 0.73829 / 0.83322, acc =  75.525 /  73.013
00:01:37 epoch: 

In [54]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

Dataset length: 6670
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
train dataset: 5336, test dataset: 1334

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 979])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-04 08:40:04 siam_strokes_ResNet10
load_model: logs/2025-05-04 08:40:04 siam_strokes_ResNet10/siam/24 epoch.pt
train/val acc : 77.43628692626953 /

#### Balanced

In [24]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

train dataset
Потанина Александра Викторовна   :   1107
Фарафонтова Таисия Михайловна   :   620
Вагнер (Дзвонкевич) Екатерина Николаевна   :   564
Крутовский Владимир Михайлович   :   542
Ивановский Алексей Осипович   :   531
Антонова Валентина Константиновна   :   435
Семидалов Вениамин Иванович   :   414
Проскурякова Е. Павловна   :   391
Крутовская Лидия Симоновна   :   369
Ржавская Надежда Федоровна   :   360

test dataset
Потанина Александра Викторовна   :   277
Фарафонтова Таисия Михайловна   :   155
Вагнер (Дзвонкевич) Екатерина Николаевна   :   142
Крутовский Владимир Михайлович   :   136
Ивановский Алексей Осипович   :   133
Антонова Валентина Константиновна   :   109
Семидалов Вениамин Иванович   :   104
Проскурякова Е. Павловна   :   98
Крутовская Лидия Симоновна   :   93
Ржавская Надежда Федоровна   :   90

train dataset: 5333, test dataset: 1337
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 994])
model:   torch.

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:23 epoch:    0, train/val loss = 7.72244 / 6.67444, acc =  65.123 /  69.858
00:00:46 epoch:    1, train/val loss = 6.59225 / 6.17835, acc =  68.742 /  69.185
00:01:10 epoch:    2, train/val loss = 6.12367 / 6.22570, acc =  71.086 /  70.381
00:01:33 epoch:    3, train/val loss = 5.51728 / 6.25998, acc =  73.767 /  68.960
00:01:56 epoch:    4, train/val loss = 5.68212 / 5.76591, acc =  72.567 /  72.102
00:02:19 epoch:    5, train/val loss = 5.60178 / 5.85153, acc =  73.130 /  71.803
00:02:43 epoch:    6, train/val loss = 5.30326 / 6.17127, acc =  75.061 /  69.559
00:03:06 epoch:    7, train/val loss = 5.21083 / 5.49194, acc =  75.755 /  73.074
00:03:29 epoch:    8, train/val loss = 5.25903 / 5.37708, acc =  75.530 /  74.420
00:03:53 epoch:    9, train/val loss = 5.14082 / 5.34339, acc =  76.074 /  74.944
00:04:16 epoch:   10, train/val loss = 5.18535 / 5.51320, acc =  75.211 /  73.373
00:04:40 epoch:   11, train/val loss = 5.09128 / 5.11535, acc =  75.923 /  75.617
00:05:03 epoch: 

...

train/val acc : 77.329833984375 / 74.94390869140625
Siam model parameters number: 0
Class model parameters number: 517642
train dataset: 5333, test dataset: 1337
--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 1081])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 10])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

00:00:07 epoch:    0, train/val loss = 1.08391 / 1.01993, acc =  65.217 /  64.398
00:00:16 epoch:    1, train/val loss = 0.95620 / 0.96689, acc =  68.779 /  67.240
00:00:24 epoch:    2, train/val loss = 0.94654 / 0.94335, acc =  68.573 /  69.484
00:00:32 epoch:    3, train/val loss = 0.91849 / 1.00053, acc =  69.773 /  66.343
00:00:40 epoch:    4, train/val loss = 0.89330 / 0.95558, acc =  69.886 /  68.362
00:00:47 epoch:    5, train/val loss = 0.86662 / 0.92472, acc =  71.273 /  70.082
00:00:55 epoch:    6, train/val loss = 0.83159 / 1.01762, acc =  72.511 /  65.744
00:01:03 epoch:    7, train/val loss = 0.85153 / 0.91773, acc =  71.592 /  70.456
00:01:10 epoch:    8, train/val loss = 0.78052 / 0.99608, acc =  74.255 /  68.138
00:01:18 epoch:    9, train/val loss = 0.82275 / 0.96594, acc =  72.736 /  66.043
00:01:25 epoch:   10, train/val loss = 0.79255 / 0.91659, acc =  73.711 /  70.381
00:01:33 epoch:   11, train/val loss = 0.80640 / 0.93944, acc =  72.698 /  69.783
00:01:41 epoch: 

In [55]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 10,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
train dataset: 5333, test dataset: 1337

--- test_pipeline:
anchor: torch.Size([64, 3, 50, 1066])
model:   torch.Size([64, 1000])
--- test_pipeline end

Find 2025-05-04 08:56:06 siam_strokes_R

### 20 class

#### Unbalanced 

In [25]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

Dataset length: 10216
Потанина Александра Викторовна   :   1384
Фарафонтова Таисия Михайловна   :   775
Вагнер (Дзвонкевич) Екатерина Николаевна   :   706
Крутовский Владимир Михайлович   :   678
Ивановский Алексей Осипович   :   664
Антонова Валентина Константиновна   :   544
Семидалов Вениамин Иванович   :   518
Проскурякова Е. Павловна   :   489
Крутовская Лидия Симоновна   :   462
Ржавская Надежда Федоровна   :   450
Мендельсон Николай Михайлович   :   426
Серошевский Вацлав Леопольдович   :   412
Попов Иван Иванович   :   404
Свентицкая Мария Хрисанфовна   :   392
Адрианов Александр Васильевич   :   382
Анучин Дмитрий Николаевич   :   369
Гуркин Григорий Иванович   :   306
Лаврский Валериан Викторович   :   288
Козьмин Николай Николаевич   :   285
Лаврский Константин Викторович   :   282

train dataset: 8172, test dataset: 2044
--- test_pipeline:
anchor: torch.Size([64, 3, 50, 1275])
model:   torch.Size([64, 1000])
--- test

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

00:00:49 epoch:    0, train/val loss = 7.14303 / 6.57589, acc =  67.768 /  69.178
00:01:28 epoch:    1, train/val loss = 6.24953 / 5.73626, acc =  69.775 /  73.434
00:02:08 epoch:    2, train/val loss = 5.83885 / 5.63071, acc =  72.455 /  73.483
00:02:48 epoch:    3, train/val loss = 5.71095 / 5.74518, acc =  72.663 /  73.141
00:03:28 epoch:    4, train/val loss = 5.47351 / 5.47556, acc =  74.082 /  74.168
00:04:08 epoch:    5, train/val loss = 5.47441 / 5.71508, acc =  73.776 /  72.358
00:04:47 epoch:    6, train/val loss = 5.61615 / 5.39016, acc =  73.042 /  73.973
00:05:27 epoch:    7, train/val loss = 5.20907 / 5.22522, acc =  75.281 /  74.853
00:06:07 epoch:    8, train/val loss = 5.41645 / 5.28887, acc =  74.437 /  74.609
00:06:47 epoch:    9, train/val loss = 5.28911 / 5.11213, acc =  74.547 /  77.006
00:07:27 epoch:   10, train/val loss = 5.16742 / 5.60729, acc =  75.710 /  72.114
00:08:08 epoch:   11, train/val loss = 5.10868 / 5.28506, acc =  76.187 /  75.196
00:08:49 epoch: 

...

train/val acc : 76.44395446777344 / 75.0
Siam model parameters number: 0
Class model parameters number: 522772
Dataset length: 10216
train dataset: 8172, test dataset: 2044

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 1000])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 20])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

00:00:12 epoch:    0, train/val loss = 1.67939 / 1.44263, acc =  49.400 /  55.577
00:00:26 epoch:    1, train/val loss = 1.43964 / 1.40050, acc =  55.947 /  56.996
00:00:39 epoch:    2, train/val loss = 1.37960 / 1.36962, acc =  57.477 /  56.996
00:00:52 epoch:    3, train/val loss = 1.32917 / 1.36418, acc =  59.068 /  57.436
00:01:05 epoch:    4, train/val loss = 1.29796 / 1.34644, acc =  59.741 /  57.779
00:01:18 epoch:    5, train/val loss = 1.27363 / 1.32645, acc =  60.646 /  58.121
00:01:32 epoch:    6, train/val loss = 1.24958 / 1.29563, acc =  61.686 /  60.470
00:01:45 epoch:    7, train/val loss = 1.21267 / 1.30395, acc =  62.506 /  59.589
00:01:58 epoch:    8, train/val loss = 1.19695 / 1.28386, acc =  62.408 /  60.714
00:02:11 epoch:    9, train/val loss = 1.17995 / 1.28024, acc =  63.815 /  60.225
00:02:24 epoch:   10, train/val loss = 1.15978 / 1.26826, acc =  63.607 /  60.861
00:02:37 epoch:   11, train/val loss = 1.13675 / 1.24999, acc =  64.391 /  60.127
00:02:50 epoch: 

In [56]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

Dataset length: 10216
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
Мендельсон Николай Михайлович                        :   426
Серошевский Вацлав Леопольдович                      :   412
Попов Иван Иванович                                  :   404
Свентицкая Мария Хрисанфовна                         :   392
Адрианов Александр Васильевич                        :   382
Ануч

#### Balanced

In [26]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

train dataset
Потанина Александра Викторовна   :   1107
Фарафонтова Таисия Михайловна   :   620
Вагнер (Дзвонкевич) Екатерина Николаевна   :   564
Крутовский Владимир Михайлович   :   542
Ивановский Алексей Осипович   :   531
Антонова Валентина Константиновна   :   435
Семидалов Вениамин Иванович   :   414
Проскурякова Е. Павловна   :   391
Крутовская Лидия Симоновна   :   369
Ржавская Надежда Федоровна   :   360
Мендельсон Николай Михайлович   :   340
Серошевский Вацлав Леопольдович   :   329
Попов Иван Иванович   :   323
Свентицкая Мария Хрисанфовна   :   313
Адрианов Александр Васильевич   :   305
Анучин Дмитрий Николаевич   :   295
Гуркин Григорий Иванович   :   244
Лаврский Валериан Викторович   :   230
Козьмин Николай Николаевич   :   228
Лаврский Константин Викторович   :   225

test dataset
Потанина Александра Викторовна   :   277
Фарафонтова Таисия Михайловна   :   155
Вагнер (Дзвонкевич) Екатерина Николаевна   :   142


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

00:00:40 epoch:    0, train/val loss = 7.18728 / 6.42552, acc =  67.557 /  70.453
00:01:21 epoch:    1, train/val loss = 6.13188 / 5.77449, acc =  70.447 /  72.989
00:02:03 epoch:    2, train/val loss = 5.78958 / 5.75461, acc =  72.652 /  72.550
00:02:43 epoch:    3, train/val loss = 5.58249 / 5.83730, acc =  73.644 /  72.355
00:03:24 epoch:    4, train/val loss = 5.56026 / 5.67717, acc =  73.166 /  72.599
00:04:05 epoch:    5, train/val loss = 5.27241 / 5.57467, acc =  74.942 /  73.135
00:04:45 epoch:    6, train/val loss = 5.32297 / 5.64587, acc =  74.856 /  72.355
00:05:26 epoch:    7, train/val loss = 5.32945 / 5.36812, acc =  74.415 /  72.794
00:06:06 epoch:    8, train/val loss = 5.16325 / 5.58566, acc =  75.885 /  73.574
00:06:45 epoch:    9, train/val loss = 5.23026 / 5.09034, acc =  75.162 /  76.451
00:07:26 epoch:   10, train/val loss = 5.21928 / 5.36787, acc =  75.175 /  74.110
00:08:06 epoch:   11, train/val loss = 5.10546 / 5.18621, acc =  75.370 /  74.939
00:08:47 epoch: 

...

train/val acc : 76.11756896972656 / 74.64651489257812
Siam model parameters number: 0
Class model parameters number: 522772
train dataset: 8165, test dataset: 2051
--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 940])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 20])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

00:00:13 epoch:    0, train/val loss = 1.77607 / 1.55464, acc =  46.258 /  51.926
00:00:26 epoch:    1, train/val loss = 1.47698 / 1.47059, acc =  54.244 /  53.730
00:00:39 epoch:    2, train/val loss = 1.43597 / 1.42937, acc =  55.199 /  57.387
00:00:52 epoch:    3, train/val loss = 1.36485 / 1.50149, acc =  57.893 /  54.803
00:01:05 epoch:    4, train/val loss = 1.35540 / 1.34997, acc =  57.832 /  58.167
00:01:19 epoch:    5, train/val loss = 1.28226 / 1.45255, acc =  60.073 /  57.143
00:01:32 epoch:    6, train/val loss = 1.25056 / 1.36835, acc =  61.310 /  61.141
00:01:46 epoch:    7, train/val loss = 1.23576 / 1.38559, acc =  61.739 /  57.972
00:01:59 epoch:    8, train/val loss = 1.21986 / 1.37507, acc =  61.739 /  58.508
00:02:12 epoch:    9, train/val loss = 1.16893 / 1.29382, acc =  62.670 /  60.166
00:02:25 epoch:   10, train/val loss = 1.17796 / 1.32082, acc =  62.964 /  60.653
00:02:38 epoch:   11, train/val loss = 1.12075 / 1.38060, acc =  64.850 /  57.143
00:02:51 epoch: 

In [57]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 20,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
Мендельсон Николай Михайлович                        :   426 = 340 + 86
Серошевский Вацлав Леопольдович                      :   412 = 329 + 83
Попов Иван Иванович                             

### 100 class

#### Unbalanced 

In [27]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

Dataset length: 20909
Потанина Александра Викторовна   :   1384
Фарафонтова Таисия Михайловна   :   775
Вагнер (Дзвонкевич) Екатерина Николаевна   :   706
Крутовский Владимир Михайлович   :   678
Ивановский Алексей Осипович   :   664
Антонова Валентина Константиновна   :   544
Семидалов Вениамин Иванович   :   518
Проскурякова Е. Павловна   :   489
Крутовская Лидия Симоновна   :   462
Ржавская Надежда Федоровна   :   450
Мендельсон Николай Михайлович   :   426
Серошевский Вацлав Леопольдович   :   412
Попов Иван Иванович   :   404
Свентицкая Мария Хрисанфовна   :   392
Адрианов Александр Васильевич   :   382
Анучин Дмитрий Николаевич   :   369
Гуркин Григорий Иванович   :   306
Лаврский Валериан Викторович   :   288
Козьмин Николай Николаевич   :   285
Лаврский Константин Викторович   :   282
Костюрина Мария Николаевна   :   258
Руднев Андрей Дмитриевич   :   258
Педашенко-Третьякова Мария Ивановна   :   239
Красноженова Мария В

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:02:02 epoch:    0, train/val loss = 6.93014 / 6.19374, acc =  67.382 /  70.301
00:03:34 epoch:    1, train/val loss = 6.03329 / 6.05707, acc =  71.005 /  71.234
00:05:03 epoch:    2, train/val loss = 5.85713 / 5.60319, acc =  71.507 /  73.840
00:06:32 epoch:    3, train/val loss = 5.90327 / 5.60856, acc =  71.507 /  73.314
00:08:01 epoch:    4, train/val loss = 5.63268 / 5.64695, acc =  72.828 /  72.262
00:09:30 epoch:    5, train/val loss = 5.62817 / 5.85519, acc =  72.924 /  71.903
00:10:58 epoch:    6, train/val loss = 5.52760 / 5.57535, acc =  73.869 /  73.529
00:12:27 epoch:    7, train/val loss = 5.60860 / 5.64944, acc =  72.990 /  72.836
00:13:56 epoch:    8, train/val loss = 5.55128 / 5.44728, acc =  73.348 /  73.434
00:15:24 epoch:    9, train/val loss = 5.35973 / 5.65639, acc =  74.293 /  73.171
00:16:53 epoch:   10, train/val loss = 5.54675 / 5.44784, acc =  73.271 /  74.055
00:18:23 epoch:   11, train/val loss = 5.47804 / 5.45974, acc =  73.749 /  72.932
00:19:51 epoch: 

...

train/val acc : 74.68164825439453 / 72.5011978149414
Siam model parameters number: 0
Class model parameters number: 563812
Dataset length: 20909
train dataset: 16727, test dataset: 4182

--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 944])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 100])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:00:29 epoch:    0, train/val loss = 3.19203 / 2.88836, acc =  25.755 /  29.723
00:00:58 epoch:    1, train/val loss = 2.78653 / 2.78267, acc =  31.673 /  31.396
00:01:27 epoch:    2, train/val loss = 2.67143 / 2.66421, acc =  33.700 /  34.864
00:01:59 epoch:    3, train/val loss = 2.59968 / 2.62942, acc =  34.704 /  34.433
00:02:31 epoch:    4, train/val loss = 2.54061 / 2.61588, acc =  35.631 /  34.075
00:03:00 epoch:    5, train/val loss = 2.48168 / 2.58555, acc =  36.827 /  35.892
00:03:29 epoch:    6, train/val loss = 2.43932 / 2.59651, acc =  37.711 /  36.107
00:03:58 epoch:    7, train/val loss = 2.41118 / 2.57352, acc =  37.831 /  36.155
00:04:27 epoch:    8, train/val loss = 2.37273 / 2.52092, acc =  38.596 /  37.279
00:04:56 epoch:    9, train/val loss = 2.34395 / 2.51222, acc =  39.009 /  37.088
00:05:25 epoch:   10, train/val loss = 2.31269 / 2.51509, acc =  40.097 /  36.657
00:05:55 epoch:   11, train/val loss = 2.28016 / 2.51733, acc =  40.067 /  37.279
00:06:26 epoch: 

In [55]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : False,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

Dataset length: 20909
Потанина Александра Викторовна                       :   1384
Фарафонтова Таисия Михайловна                        :   775
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706
Крутовский Владимир Михайлович                       :   678
Ивановский Алексей Осипович                          :   664
Антонова Валентина Константиновна                    :   544
Семидалов Вениамин Иванович                          :   518
Проскурякова Е. Павловна                             :   489
Крутовская Лидия Симоновна                           :   462
Ржавская Надежда Федоровна                           :   450
Мендельсон Николай Михайлович                        :   426
Серошевский Вацлав Леопольдович                      :   412
Попов Иван Иванович                                  :   404
Свентицкая Мария Хрисанфовна                         :   392
Адрианов Александр Васильевич                        :   382
Ануч

#### Balanced

In [28]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

train dataset
Потанина Александра Викторовна   :   1107
Фарафонтова Таисия Михайловна   :   620
Вагнер (Дзвонкевич) Екатерина Николаевна   :   564
Крутовский Владимир Михайлович   :   542
Ивановский Алексей Осипович   :   531
Антонова Валентина Константиновна   :   435
Семидалов Вениамин Иванович   :   414
Проскурякова Е. Павловна   :   391
Крутовская Лидия Симоновна   :   369
Ржавская Надежда Федоровна   :   360
Мендельсон Николай Михайлович   :   340
Серошевский Вацлав Леопольдович   :   329
Попов Иван Иванович   :   323
Свентицкая Мария Хрисанфовна   :   313
Адрианов Александр Васильевич   :   305
Анучин Дмитрий Николаевич   :   295
Гуркин Григорий Иванович   :   244
Лаврский Валериан Викторович   :   230
Козьмин Николай Николаевич   :   228
Лаврский Константин Викторович   :   225
Костюрина Мария Николаевна   :   206
Руднев Андрей Дмитриевич   :   206
Педашенко-Третьякова Мария Ивановна   :   191
Красноженова Мария Васильевн

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:01:25 epoch:    0, train/val loss = 6.94558 / 6.16085, acc =  67.837 /  70.393
00:02:56 epoch:    1, train/val loss = 6.06132 / 5.80817, acc =  70.810 /  71.720
00:04:31 epoch:    2, train/val loss = 5.80422 / 5.43260, acc =  71.924 /  74.491
00:06:05 epoch:    3, train/val loss = 5.78432 / 5.57585, acc =  72.212 /  73.946
00:07:34 epoch:    4, train/val loss = 5.72646 / 5.50433, acc =  72.541 /  73.685
00:09:01 epoch:    5, train/val loss = 5.61511 / 5.44424, acc =  73.171 /  74.088
00:10:27 epoch:    6, train/val loss = 5.54864 / 5.53976, acc =  73.704 /  72.928
00:11:56 epoch:    7, train/val loss = 5.50996 / 5.58271, acc =  73.764 /  72.430
00:13:25 epoch:    8, train/val loss = 5.44104 / 5.87905, acc =  73.800 /  72.027
00:14:52 epoch:    9, train/val loss = 5.49095 / 5.60990, acc =  73.494 /  72.193
00:16:20 epoch:   10, train/val loss = 5.40544 / 5.63997, acc =  73.962 /  72.951
00:17:46 epoch:   11, train/val loss = 5.18629 / 5.65001, acc =  75.328 /  73.117
00:19:12 epoch: 

...

train/val acc : 74.12356567382812 / 73.3301773071289
Siam model parameters number: 0
Class model parameters number: 563812
train dataset: 16687, test dataset: 4222
--- test_pipeline:
X_batch: torch.Size([64, 3, 50, 842])
y_batch: torch.Size([64])
embs: torch.Size([64, 1000])
model:   torch.Size([64, 100])
--- test_pipeline end

Training start


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

00:00:28 epoch:    0, train/val loss = 3.31101 / 3.09374, acc =  19.560 /  23.520
00:00:58 epoch:    1, train/val loss = 2.88315 / 2.91733, acc =  27.866 /  27.428
00:01:26 epoch:    2, train/val loss = 2.72462 / 2.84132, acc =  30.359 /  28.754
00:01:54 epoch:    3, train/val loss = 2.59731 / 2.82503, acc =  33.038 /  29.180
00:02:23 epoch:    4, train/val loss = 2.49499 / 2.77656, acc =  35.423 /  30.270
00:02:51 epoch:    5, train/val loss = 2.43383 / 2.78429, acc =  36.765 /  29.796
00:03:19 epoch:    6, train/val loss = 2.34302 / 2.82236, acc =  38.881 /  29.370
00:03:47 epoch:    7, train/val loss = 2.31773 / 2.70037, acc =  39.378 /  32.591
00:04:16 epoch:    8, train/val loss = 2.24022 / 2.75664, acc =  41.122 /  30.767
00:04:44 epoch:    9, train/val loss = 2.18903 / 2.79004, acc =  42.704 /  31.383
00:05:13 epoch:   10, train/val loss = 2.11767 / 2.73453, acc =  44.496 /  30.910
00:05:41 epoch:   11, train/val loss = 2.05684 / 2.75031, acc =  45.323 /  31.360
00:06:10 epoch: 

In [56]:
config = {
    "device" : DEVICE,

    "type" : "strokes_ResNet",

    "model" : "ResNet",
    "siam_emb_dim" : 1000,
    "num_not_freeze_layers" : 1,

    "dataset_topk" : 100,
    "folder" : "КККМFOLDER_strokes_bin_ok",

    "split_percent" : 0.8,
    "balanced" : True,

    "batch_size" : 64,
    "siam_epochs" : 30,
    "cl_epochs" : 30,

    "trained" : True,
}

experiment_strokes(config)

Device: cuda
Siam ResNet model parameters number: 513000

train + test dataset
Потанина Александра Викторовна                       :   1384 = 1107 + 277
Фарафонтова Таисия Михайловна                        :   775 = 620 + 155
Вагнер (Дзвонкевич) Екатерина Николаевна             :   706 = 564 + 142
Крутовский Владимир Михайлович                       :   678 = 542 + 136
Ивановский Алексей Осипович                          :   664 = 531 + 133
Антонова Валентина Константиновна                    :   544 = 435 + 109
Семидалов Вениамин Иванович                          :   518 = 414 + 104
Проскурякова Е. Павловна                             :   489 = 391 + 98
Крутовская Лидия Симоновна                           :   462 = 369 + 93
Ржавская Надежда Федоровна                           :   450 = 360 + 90
Мендельсон Николай Михайлович                        :   426 = 340 + 86
Серошевский Вацлав Леопольдович                      :   412 = 329 + 83
Попов Иван Иванович                             

# End